# From Zero to RAG: An Incremental, Hands-On Notebook

Welcome to this comprehensive tutorial on building a Retrieval-Augmented Generation (RAG) pipeline from scratch! In this notebook, you'll learn how to create a complete RAG system incrementally, starting with basic keyword search and progressing to sophisticated semantic retrieval with re-ranking.

## What You'll Learn
- Build retrieval systems using TF-IDF, BM25, and semantic embeddings
- Implement hybrid retrieval combining lexical and semantic approaches
- Apply rank fusion techniques (Reciprocal Rank Fusion)
- Use cross-encoder re-ranking for improved precision
- Create an end-to-end RAG pipeline with generation

## What Gets Built
By the end, you'll have a working RAG system that can answer questions about a synthetic knowledge base, complete with retrieval, re-ranking, and generation components.

## Technical Constraints
- **Python 3.10+** compatible code throughout
- **Open-source models only** for embeddings and re-ranking (sentence-transformers, cross-encoders)
- **OpenAI SDK** used only for the final generation step
- All dependencies installable via pip
- Deterministic behavior with fixed random seeds

In [1]:
# Setup & Environment Check
import sys
import os
import subprocess
import importlib.util

# Print Python version to verify compatibility
print(f"Python version: {sys.version}")
if sys.version_info < (3, 10):
    print("⚠️  Warning: This notebook requires Python 3.10 or higher")
else:
    print("✅ Python version compatible")

# Define required packages
required_packages = [
    'numpy',
    'pandas', 
    'scikit-learn',
    'rank_bm25',
    'sentence-transformers',
    'torch',
    'faiss-cpu',
    'tqdm',
    'openai'
]

def install_package(package_name):
    """Install a package using pip programmatically"""
    try:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package_name])
        print(f"✅ Successfully installed {package_name}")
        return True
    except subprocess.CalledProcessError:
        print(f"❌ Failed to install {package_name}")
        return False

def check_and_install_packages(packages):
    """Check if packages are available, install if missing"""
    missing_packages = []
    
    # First pass: check what's missing
    for package in packages:
        # Handle special cases for import names vs package names
        import_name = package
        if package == 'scikit-learn':
            import_name = 'sklearn'
        elif package == 'faiss-cpu':
            import_name = 'faiss'
        elif package == 'rank_bm25':
            import_name = 'rank_bm25'
            
        spec = importlib.util.find_spec(import_name)
        if spec is None:
            missing_packages.append(package)
            print(f"❌ {package} not found")
        else:
            print(f"✅ {package} available")
    
    # Second pass: install missing packages
    if missing_packages:
        print(f"\n📦 Installing {len(missing_packages)} missing packages...")
        for package in missing_packages:
            install_package(package)
    
    return missing_packages

# Check and install packages
missing = check_and_install_packages(required_packages)

print("\n🔄 Importing all packages...")
try:
    import numpy as np
    import pandas as pd
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity
    from rank_bm25 import BM25Okapi
    from sentence_transformers import SentenceTransformer, CrossEncoder
    import torch
    try:
        import faiss
        FAISS_AVAILABLE = True
    except ImportError:
        from sklearn.neighbors import NearestNeighbors
        FAISS_AVAILABLE = False
        print("ℹ️  FAISS not available, will use sklearn NearestNeighbors")
    from tqdm import tqdm
    from openai import OpenAI
    
    print("✅ All imports successful!")
    
except ImportError as e:
    print(f"❌ Import failed: {e}")
    print("Please restart the kernel and try again.")

# Set global random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)
    
print("\n🎯 Random seeds set for reproducibility")
print("🚀 Environment ready!")

Python version: 3.12.4 | packaged by Anaconda, Inc. | (main, Jun 18 2024, 10:07:17) [Clang 14.0.6 ]
✅ Python version compatible
✅ numpy available
✅ pandas available
✅ scikit-learn available
✅ rank_bm25 available
❌ sentence-transformers not found
✅ torch available
✅ faiss-cpu available
✅ tqdm available
✅ openai available

📦 Installing 1 missing packages...
✅ Successfully installed sentence-transformers

🔄 Importing all packages...
✅ All imports successful!

🎯 Random seeds set for reproducibility
🚀 Environment ready!


## What is RAG?

**Retrieval-Augmented Generation (RAG)** combines information retrieval with text generation to create more accurate, grounded responses. Instead of relying solely on a language model's training data, RAG first retrieves relevant documents from a knowledge base, then uses those documents to generate answers.

### Core Components
1. **Indexing**: Preprocessing and storing documents for efficient retrieval
2. **Retrieval**: Finding relevant documents given a query
3. **Fusion**: Combining results from multiple retrieval methods
4. **Re-ranking**: Refining the order of retrieved documents
5. **Generation**: Creating answers using retrieved context

### Key Benefits
- **Reduces hallucinations** by grounding responses in actual documents
- **Enables up-to-date information** without retraining models
- **Provides citations** for transparency and verification
- **Scales efficiently** to large knowledge bases

### Trade-offs
RAG adds complexity and latency but dramatically improves factual accuracy and allows dynamic knowledge updates. The retrieval quality directly impacts the final answer quality.

## Dataset Preparation

Quality retrieval starts with quality data. Clean, well-structured documents are essential for effective RAG systems. Each document should have consistent fields and clear, focused content.

### Key Principles
- **Structured format**: Use consistent fields (id, title, text) for easy processing
- **Appropriate granularity**: Documents should be focused but comprehensive
- **Clean text**: Remove formatting artifacts, normalize whitespace
- **Diverse content**: Include varied topics to test retrieval robustness

### Our Synthetic Corpus
We'll create a small but diverse dataset spanning multiple domains (astronomy, cooking, programming, history, health, sports). This allows us to test different retrieval methods on varied content types. Including some near-duplicates and paraphrases helps evaluate robustness to semantic similarity.

### Licensing Note
When using real data, always verify licensing terms and respect copyright. Our synthetic dataset avoids these concerns while providing realistic testing scenarios.

**Deterministic seeds** ensure reproducible results across runs, crucial for comparing retrieval methods fairly.

In [2]:
# Comprehensive synthetic dataset creation
# This dataset will be used throughout the entire notebook
import os

# Create data directory if it doesn't exist
os.makedirs('./data', exist_ok=True)

# Define synthetic documents across 6 domains with varied content types
# Each domain has 5-8 documents including some near-duplicates for robustness testing
documents = [
    # Astronomy domain
    {"id": "ast_001", "title": "Black Holes: Cosmic Vacuum Cleaners", 
     "text": "Black holes are regions of spacetime where gravity is so strong that nothing, not even light, can escape once it crosses the event horizon. They form when massive stars collapse at the end of their lives. The event horizon is the boundary beyond which escape becomes impossible. Supermassive black holes, found at galaxy centers, can have masses millions of times greater than our Sun."},
    
    {"id": "ast_002", "title": "The Life Cycle of Stars", 
     "text": "Stars begin as clouds of gas and dust that collapse under gravity. Nuclear fusion ignites in their cores, converting hydrogen to helium and releasing enormous energy. Main sequence stars like our Sun burn steadily for billions of years. Massive stars burn through their fuel quickly and end in spectacular supernovae, while smaller stars become white dwarfs."},
    
    {"id": "ast_003", "title": "Exoplanets: Worlds Beyond Our Solar System", 
     "text": "Exoplanets are planets orbiting stars other than our Sun. Over 5,000 have been discovered using methods like transit photometry and radial velocity measurements. Some exoplanets orbit in their star's habitable zone where liquid water could exist. The Kepler Space Telescope revolutionized exoplanet discovery by monitoring stellar brightness changes."},
    
    {"id": "ast_004", "title": "The Expanding Universe and Dark Energy", 
     "text": "The universe is expanding, with distant galaxies moving away from us at speeds proportional to their distance. This expansion is accelerating due to dark energy, a mysterious force comprising about 68% of the universe. Dark matter, another enigma, makes up 27%, leaving only 5% as ordinary matter we can see and touch."},
    
    {"id": "ast_005", "title": "Stellar Collapse and Black Hole Formation", 
     "text": "When massive stars exhaust their nuclear fuel, they can no longer support themselves against gravity. The core collapses in milliseconds, creating conditions so extreme that a black hole forms. The collapsing matter crushes down to infinite density at the singularity, while the event horizon marks the point of no return for any approaching matter or light."},
    
    # Cooking domain  
    {"id": "cook_001", "title": "The Maillard Reaction: Science of Browning", 
     "text": "The Maillard reaction occurs when proteins and sugars are heated together, creating complex flavors and browning in foods. This reaction is responsible for the golden crust on bread, the sear on steaks, and the rich flavor of roasted coffee. Temperature control is crucial - too low and the reaction won't occur, too high and food burns before proper flavor development."},
    
    {"id": "cook_002", "title": "Knife Skills: Foundation of Good Cooking", 
     "text": "Proper knife technique is essential for efficient cooking. A sharp knife is safer than a dull one because it requires less pressure and provides better control. The basic cuts include julienne, dice, chiffonade, and brunoise. Consistent sizing ensures even cooking. Always curl your fingers and use a rocking motion with the knife tip staying on the cutting board."},
    
    {"id": "cook_003", "title": "Understanding Heat Transfer in Cooking", 
     "text": "Heat transfer in cooking occurs through conduction, convection, and radiation. Conduction happens when food touches a hot surface like a pan. Convection involves hot air or liquid circulating around food. Radiation transfers heat through electromagnetic waves, like in broiling. Understanding these principles helps choose the right cooking method for desired results."},
    
    {"id": "cook_004", "title": "Fermentation: Ancient Preservation Technique", 
     "text": "Fermentation uses beneficial microorganisms to preserve food and create unique flavors. Lactic acid fermentation produces sauerkraut and kimchi, while alcoholic fermentation creates wine and beer. The process requires controlling temperature, pH, and salt levels to favor good bacteria over harmful ones. Fermented foods are also rich in probiotics."},
    
    {"id": "cook_005", "title": "Browning and Caramelization in Food", 
     "text": "Browning reactions transform food appearance and flavor through heat. Caramelization breaks down sugars at high temperatures, creating sweet, complex flavors. The Maillard reaction between amino acids and sugars produces savory, nutty notes. Both reactions require precise temperature control to achieve desired results without burning."},
    
    # Python/Programming domain
    {"id": "py_001", "title": "Python List Comprehensions: Elegant Iteration", 
     "text": "List comprehensions provide a concise way to create lists in Python. They're more readable and often faster than traditional for loops. The syntax is [expression for item in iterable if condition]. For example, [x**2 for x in range(10) if x%2==0] creates squares of even numbers. Nested comprehensions are possible but should be used sparingly for readability."},
    
    {"id": "py_002", "title": "Python Decorators: Modifying Function Behavior", 
     "text": "Decorators are a powerful Python feature that allows modifying or extending function behavior without changing the function itself. They use the @decorator_name syntax above function definitions. Common uses include logging, timing, authentication, and caching. Decorators are functions that take functions as arguments and return modified functions."},
    
    {"id": "py_003", "title": "Virtual Environments: Dependency Management", 
     "text": "Virtual environments create isolated Python installations for different projects, preventing dependency conflicts. Tools like venv, conda, and pipenv help manage environments. Each environment has its own Python interpreter and package installations. This isolation ensures that project dependencies don't interfere with each other or the system Python."},
    
    {"id": "py_004", "title": "Exception Handling: Graceful Error Management", 
     "text": "Python's try-except blocks handle errors gracefully without crashing programs. Different exception types can be caught specifically, and finally blocks ensure cleanup code runs regardless of errors. Raising custom exceptions helps create more informative error messages. Proper exception handling makes code more robust and user-friendly."},
    
    {"id": "py_005", "title": "Python Generators: Memory-Efficient Iteration", 
     "text": "Generators create iterators that yield values on-demand rather than storing them in memory. They use the yield keyword instead of return. Generator expressions use parentheses instead of square brackets like list comprehensions. This lazy evaluation saves memory for large datasets and enables infinite sequences."},
    
    {"id": "py_006", "title": "List Processing in Python", 
     "text": "Python offers multiple ways to process lists efficiently. List comprehensions create new lists with concise syntax: [x*2 for x in numbers]. The map() function applies operations to all elements, while filter() selects elements meeting criteria. These functional approaches are often more readable than traditional loops and can be combined for complex transformations."},
    
    # History domain
    {"id": "hist_001", "title": "The Fall of Constantinople 1453", 
     "text": "The fall of Constantinople in 1453 marked the end of the Byzantine Empire and the beginning of Ottoman dominance in southeastern Europe. Sultan Mehmed II's forces used massive cannons to breach the city's ancient walls after a 53-day siege. This event closed the eastern trade routes, spurring European exploration of new paths to Asia."},
    
    {"id": "hist_002", "title": "The Printing Press Revolution", 
     "text": "Johannes Gutenberg's printing press, invented around 1440, revolutionized the spread of information. It enabled mass production of books, making knowledge accessible beyond the wealthy elite. The printing press facilitated the Protestant Reformation, scientific revolution, and Renaissance by allowing rapid dissemination of ideas across Europe."},
    
    {"id": "hist_003", "title": "The Silk Road: Ancient Trade Networks", 
     "text": "The Silk Road was a network of trade routes connecting East and West from ancient times through the medieval period. It facilitated exchange of goods, ideas, and cultures between China, Central Asia, and Europe. Merchants traded silk, spices, precious metals, and technologies, while also spreading religions and knowledge across continents."},
    
    {"id": "hist_004", "title": "The Industrial Revolution's Impact", 
     "text": "The Industrial Revolution transformed society from agricultural to manufacturing-based economies. Steam engines, textile mills, and factory systems changed how people lived and worked. Urbanization accelerated as people moved to cities for factory jobs. This period laid the foundation for modern capitalism and dramatically increased production capacity."},
    
    {"id": "hist_005", "title": "Medieval Trade Routes and Commerce", 
     "text": "Medieval trade networks connected distant regions through overland and maritime routes. The Silk Road linked Asia and Europe, while Mediterranean trade connected Africa, Asia, and Europe. Merchants established trade guilds for protection and standardization. These routes facilitated not only commerce but also cultural and technological exchange between civilizations."},
    
    # Health domain
    {"id": "health_001", "title": "The Immune System: Body's Defense Network", 
     "text": "The immune system protects the body from pathogens through innate and adaptive responses. White blood cells, antibodies, and specialized organs work together to identify and eliminate threats. Vaccines train the immune system to recognize specific pathogens without causing illness. A healthy lifestyle supports immune function through proper nutrition, exercise, and sleep."},
    
    {"id": "health_002", "title": "Cardiovascular Health and Exercise", 
     "text": "Regular exercise strengthens the heart muscle and improves circulation throughout the body. Aerobic activities like walking, swimming, and cycling reduce blood pressure and cholesterol levels. Exercise also increases HDL (good) cholesterol while lowering LDL (bad) cholesterol. The American Heart Association recommends 150 minutes of moderate exercise weekly."},
    
    {"id": "health_003", "title": "Nutrition: Fuel for the Human Body", 
     "text": "Proper nutrition provides energy and essential nutrients for optimal body function. Macronutrients include carbohydrates for energy, proteins for tissue repair, and fats for hormone production. Micronutrients like vitamins and minerals support various biochemical processes. A balanced diet includes diverse foods from all food groups in appropriate proportions."},
    
    {"id": "health_004", "title": "Sleep: The Foundation of Health", 
     "text": "Sleep is crucial for physical and mental health, allowing the body to repair and consolidate memories. Adults need 7-9 hours of quality sleep nightly. During sleep, the brain clears metabolic waste and strengthens neural connections. Sleep deprivation impairs immune function, cognitive performance, and emotional regulation."},
    
    {"id": "health_005", "title": "Mental Health and Well-being", 
     "text": "Mental health encompasses emotional, psychological, and social well-being. It affects how we think, feel, and act. Factors like genetics, brain chemistry, trauma, and life experiences influence mental health. Regular exercise, social connections, stress management, and professional support when needed all contribute to maintaining good mental health."},
    
    # Sports domain
    {"id": "sport_001", "title": "Biomechanics of Athletic Performance", 
     "text": "Biomechanics analyzes human movement to optimize athletic performance and prevent injuries. It studies forces, motion, and energy transfer during sports activities. Understanding lever systems, joint angles, and muscle activation patterns helps athletes improve technique. Video analysis and force plates provide detailed biomechanical data for performance enhancement."},
    
    {"id": "sport_002", "title": "Sports Psychology: Mental Game", 
     "text": "Sports psychology focuses on the mental aspects of athletic performance. Techniques like visualization, goal setting, and mindfulness help athletes manage pressure and maintain focus. Mental training is as important as physical preparation. Confidence, motivation, and concentration significantly impact performance outcomes in competitive sports."},
    
    {"id": "sport_003", "title": "Recovery and Regeneration in Athletics", 
     "text": "Recovery is essential for athletic adaptation and performance improvement. It includes active recovery, proper nutrition, hydration, and sleep. Techniques like ice baths, compression therapy, and massage aid muscle recovery. Periodized training incorporates rest phases to prevent overtraining and reduce injury risk while maximizing performance gains."},
    
    {"id": "sport_004", "title": "Strength Training Principles", 
     "text": "Effective strength training follows progressive overload, gradually increasing resistance to stimulate adaptation. Compound exercises like squats and deadlifts work multiple muscle groups efficiently. Training frequency, volume, and intensity must be balanced for optimal results. Proper form prevents injury and ensures targeted muscle activation during resistance exercises."},
    
    {"id": "sport_005", "title": "Endurance Training Methodologies", 
     "text": "Endurance training improves the body's ability to sustain prolonged physical activity. Training zones based on heart rate or power output optimize different energy systems. Base training builds aerobic capacity, while high-intensity intervals improve lactate threshold. Periodization varies training stress to promote adaptation while preventing burnout and overtraining syndrome."}
]

# Convert to pandas DataFrame for easy manipulation
corpus_df = pd.DataFrame(documents)

print(f"📊 Created synthetic corpus with {len(corpus_df)} documents")
print(f"📂 Domains covered: {len(corpus_df['id'].str[:4].unique())} unique prefixes")
print(f"📏 Text length range: {corpus_df['text'].str.len().min()} - {corpus_df['text'].str.len().max()} characters")

# Show distribution by domain
domain_counts = corpus_df['id'].str[:4].value_counts()
print("\n📈 Documents per domain:")
domain_names = {
    'ast_': 'Astronomy',
    'cook': 'Cooking', 
    'py_0': 'Python/Programming',
    'hist': 'History',
    'heal': 'Health',
    'spor': 'Sports'
}
for prefix, count in domain_counts.items():
    domain_name = domain_names.get(prefix, prefix)
    print(f"  {domain_name}: {count} documents")

# Save to CSV for reuse throughout the notebook
corpus_df.to_csv('./data/corpus.csv', index=False)
print("\n💾 Corpus saved to ./data/corpus.csv")

# Display sample documents
print("\n📋 Sample documents:")
for i in range(3):
    doc = corpus_df.iloc[i]
    print(f"\n[{doc['id']}] {doc['title']}")
    print(f"Text preview: {doc['text'][:100]}...")

📊 Created synthetic corpus with 31 documents
📂 Domains covered: 6 unique prefixes
📏 Text length range: 313 - 385 characters

📈 Documents per domain:
  Python/Programming: 6 documents
  Astronomy: 5 documents
  Cooking: 5 documents
  History: 5 documents
  Health: 5 documents
  Sports: 5 documents

💾 Corpus saved to ./data/corpus.csv

📋 Sample documents:

[ast_001] Black Holes: Cosmic Vacuum Cleaners
Text preview: Black holes are regions of spacetime where gravity is so strong that nothing, not even light, can es...

[ast_002] The Life Cycle of Stars
Text preview: Stars begin as clouds of gas and dust that collapse under gravity. Nuclear fusion ignites in their c...

[ast_003] Exoplanets: Worlds Beyond Our Solar System
Text preview: Exoplanets are planets orbiting stars other than our Sun. Over 5,000 have been discovered using meth...


## TF-IDF: Term Frequency-Inverse Document Frequency

**TF-IDF** is a fundamental text retrieval technique that scores documents based on term importance. It combines two concepts:
- **Term Frequency (TF)**: How often a term appears in a document
- **Inverse Document Frequency (IDF)**: How rare a term is across the entire corpus

### Intuition
Words that appear frequently in a document but rarely across the corpus are most important for distinguishing that document. Common words like "the" and "and" get low scores, while specific terms get higher scores.

### How It Works
Documents are converted to vectors where each dimension represents a unique term's TF-IDF score. Query similarity is computed using cosine similarity between the query vector and document vectors.

### Advantages
- **Fast and interpretable**: Clear scoring rationale
- **No training required**: Works immediately on any corpus
- **Memory efficient**: Sparse vectors for large vocabularies

### Limitations
- **Vocabulary mismatch**: Can't match synonyms ("car" vs "automobile")
- **Word order ignored**: "dog bites man" = "man bites dog"
- **No semantic understanding**: Relies purely on exact word matches

In [3]:
# TF-IDF retrieval implementation using scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Initialize TF-IDF vectorizer with optimized parameters
# - Use both unigrams and bigrams to capture phrases like "black holes"
# - Convert to lowercase for normalization
# - Remove English stop words to focus on meaningful terms
# - Set max_features to control vocabulary size and computation
tfidf_vectorizer = TfidfVectorizer(
    ngram_range=(1, 2),  # Include both single words and word pairs
    lowercase=True,      # Normalize case
    stop_words='english', # Remove common words like 'the', 'and'
    max_features=10000,  # Limit vocabulary size for efficiency
    min_df=1,            # Include terms that appear in at least 1 document
    max_df=0.8           # Exclude terms that appear in >80% of documents
)

# Fit the vectorizer on our corpus and transform texts to TF-IDF vectors
print("🔄 Building TF-IDF matrix...")
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus_df['text'])

print(f"📊 TF-IDF matrix shape: {tfidf_matrix.shape}")
print(f"📝 Vocabulary size: {len(tfidf_vectorizer.vocabulary_)}")
print(f"💾 Matrix sparsity: {(1 - tfidf_matrix.nnz / tfidf_matrix.size) * 100:.1f}% zeros")

def query_tfidf(query_text, top_k=5):
    """
    Retrieve documents using TF-IDF similarity.
    
    Args:
        query_text (str): The search query
        top_k (int): Number of top results to return
    
    Returns:
        list: Tuples of (document_index, similarity_score, document_info)
    """
    # Transform query using the same vectorizer fitted on corpus
    query_vector = tfidf_vectorizer.transform([query_text])
    
    # Compute cosine similarity between query and all documents
    # Cosine similarity ranges from 0 (no similarity) to 1 (identical)
    similarity_scores = cosine_similarity(query_vector, tfidf_matrix).flatten()
    
    # Get indices of top-k most similar documents
    top_indices = similarity_scores.argsort()[-top_k:][::-1]
    
    # Build results with document info and scores
    results = []
    for idx in top_indices:
        doc_info = {
            'id': corpus_df.iloc[idx]['id'],
            'title': corpus_df.iloc[idx]['title'],
            'text': corpus_df.iloc[idx]['text']
        }
        results.append((idx, similarity_scores[idx], doc_info))
    
    return results

# Test TF-IDF retrieval with example queries
test_queries = [
    "black holes and event horizons",
    "python programming decorators"
]

print("\n🔍 Testing TF-IDF retrieval:")
for query in test_queries:
    print(f"\n📋 Query: '{query}'")
    results = query_tfidf(query, top_k=5)
    
    print("Top 5 results:")
    for rank, (idx, score, doc_info) in enumerate(results, 1):
        print(f"  {rank}. [{doc_info['id']}] {doc_info['title']} (score: {score:.3f})")
        # Show first 100 characters of text as snippet
        snippet = doc_info['text'][:100] + '...' if len(doc_info['text']) > 100 else doc_info['text']
        print(f"      {snippet}")

🔄 Building TF-IDF matrix...
📊 TF-IDF matrix shape: (31, 1734)
📝 Vocabulary size: 1734
💾 Matrix sparsity: 0.0% zeros

🔍 Testing TF-IDF retrieval:

📋 Query: 'black holes and event horizons'
Top 5 results:
  1. [ast_001] Black Holes: Cosmic Vacuum Cleaners (score: 0.445)
      Black holes are regions of spacetime where gravity is so strong that nothing, not even light, can es...
  2. [ast_005] Stellar Collapse and Black Hole Formation (score: 0.103)
      When massive stars exhaust their nuclear fuel, they can no longer support themselves against gravity...
  3. [hist_001] The Fall of Constantinople 1453 (score: 0.043)
      The fall of Constantinople in 1453 marked the end of the Byzantine Empire and the beginning of Ottom...
  4. [py_005] Python Generators: Memory-Efficient Iteration (score: 0.000)
      Generators create iterators that yield values on-demand rather than storing them in memory. They use...
  5. [ast_002] The Life Cycle of Stars (score: 0.000)
      Stars begin as clouds

## BM25: Best Matching 25

**BM25** is a probabilistic ranking function that often outperforms TF-IDF for information retrieval. It improves upon TF-IDF by addressing two key limitations:

### Key Improvements
1. **Term Saturation**: TF-IDF scores increase linearly with term frequency, but BM25 uses a saturation function. After a certain point, additional occurrences contribute less to the score.

2. **Document Length Normalization**: BM25 adjusts for document length, preventing longer documents from having unfair advantages simply due to more term occurrences.

### Parameters
- **k1** (typically 1.2-2.0): Controls term frequency saturation
- **b** (typically 0.75): Controls document length normalization strength

### When BM25 Excels
BM25 typically outperforms TF-IDF for keyword search, especially with:
- Varied document lengths
- Collections where term frequency patterns matter
- Traditional information retrieval tasks

Both TF-IDF and BM25 are **lexical** methods—they rely on exact word matches and don't understand semantics.

In [4]:
# BM25 retrieval implementation using rank_bm25
from rank_bm25 import BM25Okapi
import string
import re

def simple_tokenizer(text):
    """
    Simple tokenizer for BM25 preprocessing.
    Converts to lowercase, removes punctuation, and splits on whitespace.
    
    Args:
        text (str): Input text to tokenize
    
    Returns:
        list: List of tokens
    """
    # Convert to lowercase for case-insensitive matching
    text = text.lower()
    
    # Remove punctuation using regex (keeps alphanumeric and spaces)
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)
    
    # Split on whitespace and filter empty strings
    tokens = [token for token in text.split() if token]
    
    return tokens

# Tokenize all documents in the corpus for BM25
print("🔄 Tokenizing corpus for BM25...")
tokenized_corpus = [simple_tokenizer(doc_text) for doc_text in corpus_df['text']]

# Initialize BM25 with default parameters (k1=1.2, b=0.75)
# These are well-tested values that work well across many domains
bm25 = BM25Okapi(tokenized_corpus)

print(f"📊 BM25 index built for {len(tokenized_corpus)} documents")
print(f"📝 Average document length: {np.mean([len(doc) for doc in tokenized_corpus]):.1f} tokens")

def query_bm25(query_text, top_k=5):
    """
    Retrieve documents using BM25 scoring.
    
    Args:
        query_text (str): The search query
        top_k (int): Number of top results to return
    
    Returns:
        list: Tuples of (document_index, bm25_score, document_info)
    """
    # Tokenize query using same tokenizer as corpus
    query_tokens = simple_tokenizer(query_text)
    
    # Get BM25 scores for all documents
    # Higher scores indicate better matches
    bm25_scores = bm25.get_scores(query_tokens)
    
    # Get indices of top-k highest scoring documents
    top_indices = np.argsort(bm25_scores)[-top_k:][::-1]
    
    # Build results with document info and scores
    results = []
    for idx in top_indices:
        doc_info = {
            'id': corpus_df.iloc[idx]['id'],
            'title': corpus_df.iloc[idx]['title'],
            'text': corpus_df.iloc[idx]['text']
        }
        results.append((idx, bm25_scores[idx], doc_info))
    
    return results

# Compare BM25 vs TF-IDF on the same queries
comparison_queries = [
    "black holes event horizon",
    "python decorators function behavior",
    "exercise cardiovascular health"
]

print("\n🔍 Comparing BM25 vs TF-IDF retrieval:")
for query in comparison_queries:
    print(f"\n📋 Query: '{query}'")
    
    # Get results from both methods
    bm25_results = query_bm25(query, top_k=3)
    tfidf_results = query_tfidf(query, top_k=3)
    
    print("\n🏆 BM25 Top 3:")
    for rank, (idx, score, doc_info) in enumerate(bm25_results, 1):
        print(f"  {rank}. [{doc_info['id']}] {doc_info['title']} (BM25: {score:.2f})")
    
    print("\n📊 TF-IDF Top 3:")
    for rank, (idx, score, doc_info) in enumerate(tfidf_results, 1):
        print(f"  {rank}. [{doc_info['id']}] {doc_info['title']} (TF-IDF: {score:.3f})")
    
    # Show overlap between methods
    bm25_ids = {doc_info['id'] for _, _, doc_info in bm25_results}
    tfidf_ids = {doc_info['id'] for _, _, doc_info in tfidf_results}
    overlap = bm25_ids.intersection(tfidf_ids)
    print(f"\n🔗 Overlap: {len(overlap)}/3 documents match between methods")

🔄 Tokenizing corpus for BM25...
📊 BM25 index built for 31 documents
📝 Average document length: 50.1 tokens

🔍 Comparing BM25 vs TF-IDF retrieval:

📋 Query: 'black holes event horizon'

🏆 BM25 Top 3:
  1. [ast_001] Black Holes: Cosmic Vacuum Cleaners (BM25: 13.18)
  2. [ast_005] Stellar Collapse and Black Hole Formation (BM25: 6.68)
  3. [hist_001] The Fall of Constantinople 1453 (BM25: 1.98)

📊 TF-IDF Top 3:
  1. [ast_001] Black Holes: Cosmic Vacuum Cleaners (TF-IDF: 0.537)
  2. [ast_005] Stellar Collapse and Black Hole Formation (TF-IDF: 0.179)
  3. [hist_001] The Fall of Constantinople 1453 (TF-IDF: 0.036)

🔗 Overlap: 3/3 documents match between methods

📋 Query: 'python decorators function behavior'

🏆 BM25 Top 3:
  1. [py_002] Python Decorators: Modifying Function Behavior (BM25: 11.77)
  2. [py_006] List Processing in Python (BM25: 3.07)
  3. [py_003] Virtual Environments: Dependency Management (BM25: 2.66)

📊 TF-IDF Top 3:
  1. [py_002] Python Decorators: Modifying Function Behav

## Embeddings: Semantic Vector Representations

**Embeddings** represent text as dense vectors in high-dimensional space where semantically similar texts are close together. Unlike lexical methods (TF-IDF, BM25), embeddings can match concepts even with different vocabulary.

### Key Advantages
- **Semantic understanding**: Matches "car" with "automobile"
- **Cross-lingual capability**: Can work across languages
- **Context awareness**: Considers word relationships and context

### Vector Similarity
Cosine similarity measures the angle between vectors, ranging from -1 to 1. Values closer to 1 indicate higher semantic similarity.

### Trade-offs
- **Computational cost**: Embedding models require more resources
- **Model dependence**: Quality depends on training data and architecture
- **Interpretability**: Harder to understand why documents match

### Approximate Nearest Neighbors (ANN)
For large corpora, exact similarity search becomes slow. ANN algorithms like FAISS provide fast approximate search with minimal accuracy loss.

**Privacy note**: Using local models keeps data on your machine, unlike API-based embedding services.

In [10]:
# Text chunking for better embedding performance
# Chunking breaks long documents into smaller, focused pieces
# This improves embedding quality and allows more precise retrieval

def chunk_text(text, chunk_size_words=180, overlap_words=30):
    """
    Split text into overlapping chunks of specified word count.
    Overlap helps maintain context across chunk boundaries.
    
    Args:
        text (str): Input text to chunk
        chunk_size_words (int): Target words per chunk
        overlap_words (int): Words to overlap between chunks
    
    Returns:
        list: List of text chunks
    """
    words = text.split()
    
    # If text is shorter than chunk size, return as single chunk
    if len(words) <= chunk_size_words:
        return [text]
    
    chunks = []
    start = 0
    
    while start < len(words):
        # Define chunk end, ensuring we don't exceed word count
        end = min(start + chunk_size_words, len(words))
        
        # Extract chunk and join words back to text
        chunk_words = words[start:end]
        chunk_text = ' '.join(chunk_words)
        chunks.append(chunk_text)
        
        # Move start position, accounting for overlap
        # If this is the last chunk, break to avoid infinite loop
        if end >= len(words):
            break
        start = end - overlap_words
    
    return chunks

# Create chunked corpus for better embedding performance
print("🔄 Creating chunked corpus...")
chunked_data = []

for _, row in corpus_df.iterrows():
    doc_chunks = chunk_text(row['text'], chunk_size_words=180, overlap_words=30)
    
    for chunk_idx, text_chunk in enumerate(doc_chunks):
        chunked_data.append({
            'doc_id': row['id'],
            'chunk_id': f"{row['id']}_chunk_{chunk_idx}",
            'title': row['title'],
            'chunk_text': text_chunk
        })

# Convert to DataFrame for easy manipulation
chunked_corpus = pd.DataFrame(chunked_data)

print(f"📊 Created {len(chunked_corpus)} chunks from {len(corpus_df)} documents")
print(f"📏 Average chunk length: {chunked_corpus['chunk_text'].str.split().str.len().mean():.1f} words")

# Show chunk length distribution
chunk_lengths = chunked_corpus['chunk_text'].str.split().str.len()
print(f"📈 Chunk length distribution:")
print(f"  Min: {chunk_lengths.min()} words")
print(f"  Max: {chunk_lengths.max()} words")
print(f"  Median: {chunk_lengths.median():.1f} words")

# Show example of chunking
sample_doc = corpus_df.iloc[0]
sample_chunks = chunk_text(sample_doc['text'])
print(f"\n📋 Example chunking for '{sample_doc['title']}':")
print(f"Original text ({len(sample_doc['text'].split())} words):")
print(f"  {sample_doc['text'][:150]}...")
print(f"\nChunks created: {len(sample_chunks)}")
for i, chunk in enumerate(sample_chunks):
    print(f"  Chunk {i+1} ({len(chunk.split())} words): {chunk[:100]}...")

🔄 Creating chunked corpus...
📊 Created 31 chunks from 31 documents
📏 Average chunk length: 49.4 words
📈 Chunk length distribution:
  Min: 43 words
  Max: 64 words
  Median: 48.0 words

📋 Example chunking for 'Black Holes: Cosmic Vacuum Cleaners':
Original text (64 words):
  Black holes are regions of spacetime where gravity is so strong that nothing, not even light, can escape once it crosses the event horizon. They form ...

Chunks created: 1
  Chunk 1 (64 words): Black holes are regions of spacetime where gravity is so strong that nothing, not even light, can es...


In [11]:
# Semantic embeddings using open-source sentence-transformers
from sentence_transformers import SentenceTransformer
import os

# Use a high-quality, lightweight open-source embedding model
# all-MiniLM-L6-v2 provides good performance with reasonable speed
model_name = "sentence-transformers/all-MiniLM-L6-v2"
# Alternative models (commented for reference):
# model_name = "BAAI/bge-small-en-v1.5"  # Better quality, slightly larger
# model_name = "intfloat/e5-small-v2"     # Good multilingual support

print(f"🤖 Loading embedding model: {model_name}")
embedding_model = SentenceTransformer(model_name)

print(f"📐 Model produces {embedding_model.get_sentence_embedding_dimension()}-dimensional vectors")

# Check if embeddings already exist to avoid recomputation
embeddings_file = './data/embeddings.npz'
chunks_file = './data/chunked_corpus.csv'

if os.path.exists(embeddings_file) and os.path.exists(chunks_file):
    print("📂 Loading pre-computed embeddings...")
    embeddings_data = np.load(embeddings_file)
    chunk_embeddings = embeddings_data['embeddings']
    chunked_corpus = pd.read_csv(chunks_file)
    print(f"✅ Loaded {len(chunk_embeddings)} embeddings from cache")
else:
    # Compute embeddings for all chunks with progress bar
    print(f"🔄 Computing embeddings for {len(chunked_corpus)} chunks...")
    
    # Use tqdm for progress tracking during embedding computation
    chunk_texts = chunked_corpus['chunk_text'].tolist()
    
    # sentence-transformers handles batching internally for efficiency
    chunk_embeddings = embedding_model.encode(
        chunk_texts, 
        batch_size=32,          # Process in batches for memory efficiency
        show_progress_bar=True, # Show progress during computation
        convert_to_numpy=True   # Return as numpy array
    )
    
    # Save embeddings and chunks for future use
    np.savez_compressed(embeddings_file, embeddings=chunk_embeddings)
    chunked_corpus.to_csv(chunks_file, index=False)
    print(f"💾 Saved {len(chunk_embeddings)} embeddings to {embeddings_file}")

# Normalize embeddings for cosine similarity using dot product
# This makes cosine similarity equivalent to dot product, which is faster
from sklearn.preprocessing import normalize
chunk_embeddings_normalized = normalize(chunk_embeddings, norm='l2')

print(f"📊 Embedding matrix shape: {chunk_embeddings.shape}")
print(f"🎯 Embeddings normalized for cosine similarity")

# Choose between FAISS and sklearn based on availability
if FAISS_AVAILABLE:
    # Use FAISS for fast approximate nearest neighbor search
    print("🚀 Using FAISS for fast similarity search")
    
    # Create FAISS index for inner product (equivalent to cosine with normalized vectors)
    embedding_dim = chunk_embeddings_normalized.shape[1]
    faiss_index = faiss.IndexFlatIP(embedding_dim)  # Inner Product index
    
    # Add embeddings to index
    faiss_index.add(chunk_embeddings_normalized.astype(np.float32))
    
    # Save FAISS index
    faiss_index_file = './data/faiss.index'
    faiss.write_index(faiss_index, faiss_index_file)
    print(f"💾 FAISS index saved to {faiss_index_file}")
    
    search_backend = 'faiss'
    
else:
    # Use sklearn NearestNeighbors as fallback
    print("📚 Using sklearn NearestNeighbors for similarity search")
    from sklearn.neighbors import NearestNeighbors
    
    # Create sklearn nearest neighbors index
    nn_index = NearestNeighbors(
        n_neighbors=20,      # Maximum neighbors to consider
        metric='cosine',     # Use cosine similarity
        algorithm='brute'    # Exact search for small datasets
    )
    nn_index.fit(chunk_embeddings)
    
    search_backend = 'sklearn'

def embed_query(query_text):
    """
    Convert query text to embedding vector.
    
    Args:
        query_text (str): The search query
    
    Returns:
        np.ndarray: Query embedding vector
    """
    query_embedding = embedding_model.encode([query_text], convert_to_numpy=True)
    return normalize(query_embedding, norm='l2')[0]  # Normalize and return single vector

def semantic_search(query_text, top_k=10):
    """
    Search for semantically similar chunks using embeddings.
    
    Args:
        query_text (str): The search query
        top_k (int): Number of top results to return
    
    Returns:
        list: Tuples of (chunk_index, similarity_score, chunk_info)
    """
    # Get query embedding
    query_embedding = embed_query(query_text)
    
    if search_backend == 'faiss':
        # Use FAISS for fast search
        scores, indices = faiss_index.search(
            query_embedding.reshape(1, -1).astype(np.float32), 
            top_k
        )
        
        results = []
        for i, (idx, score) in enumerate(zip(indices[0], scores[0])):
            chunk_info = {
                'chunk_id': chunked_corpus.iloc[idx]['chunk_id'],
                'doc_id': chunked_corpus.iloc[idx]['doc_id'],
                'title': chunked_corpus.iloc[idx]['title'],
                'chunk_text': chunked_corpus.iloc[idx]['chunk_text']
            }
            results.append((idx, score, chunk_info))
    
    else:
        # Use sklearn for search
        distances, indices = nn_index.kneighbors(
            query_embedding.reshape(1, -1), 
            n_neighbors=min(top_k, len(chunked_corpus))
        )
        
        results = []
        for i, (idx, distance) in enumerate(zip(indices[0], distances[0])):
            # Convert cosine distance to similarity (1 - distance)
            similarity = 1 - distance
            chunk_info = {
                'chunk_id': chunked_corpus.iloc[idx]['chunk_id'],
                'doc_id': chunked_corpus.iloc[idx]['doc_id'],
                'title': chunked_corpus.iloc[idx]['title'],
                'chunk_text': chunked_corpus.iloc[idx]['chunk_text']
            }
            results.append((idx, similarity, chunk_info))
    
    return results

# Test semantic search
test_queries = [
    "stellar collapse and gravitational effects",
    "modifying function behavior in programming",
    "heart health and physical activity"
]

print("\n🔍 Testing semantic search:")
for query in test_queries:
    print(f"\n📋 Query: '{query}'")
    results = semantic_search(query, top_k=5)
    
    print("Top 5 semantic matches:")
    for rank, (idx, score, chunk_info) in enumerate(results, 1):
        print(f"  {rank}. [{chunk_info['doc_id']}] {chunk_info['title']} (similarity: {score:.3f})")
        snippet = chunk_info['chunk_text'][:100] + '...'
        print(f"      {snippet}")

🤖 Loading embedding model: sentence-transformers/all-MiniLM-L6-v2
📐 Model produces 384-dimensional vectors
🔄 Computing embeddings for 31 chunks...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

💾 Saved 31 embeddings to ./data/embeddings.npz
📊 Embedding matrix shape: (31, 384)
🎯 Embeddings normalized for cosine similarity
🚀 Using FAISS for fast similarity search
💾 FAISS index saved to ./data/faiss.index

🔍 Testing semantic search:

📋 Query: 'stellar collapse and gravitational effects'
Top 5 semantic matches:
  1. [ast_005] Stellar Collapse and Black Hole Formation (similarity: 0.595)
      When massive stars exhaust their nuclear fuel, they can no longer support themselves against gravity...
  2. [ast_002] The Life Cycle of Stars (similarity: 0.498)
      Stars begin as clouds of gas and dust that collapse under gravity. Nuclear fusion ignites in their c...
  3. [ast_001] Black Holes: Cosmic Vacuum Cleaners (similarity: 0.376)
      Black holes are regions of spacetime where gravity is so strong that nothing, not even light, can es...
  4. [ast_004] The Expanding Universe and Dark Energy (similarity: 0.252)
      The universe is expanding, with distant galaxies moving away fro

## Hybrid Retrieval: Best of Both Worlds

**Hybrid retrieval** combines lexical (BM25) and semantic (embeddings) approaches to leverage their complementary strengths:

### Lexical Strengths
- Exact term matching for technical terms and proper names
- Fast computation and interpretable results
- Robust to domain shifts

### Semantic Strengths
- Conceptual matching beyond exact words
- Better handling of synonyms and paraphrases
- Context-aware understanding

### Hybrid Strategy
1. **Union approach**: Get candidates from both methods
2. **Score normalization**: Make scores comparable across methods
3. **Rank fusion**: Combine rankings intelligently

### Score Normalization
Different retrieval methods produce scores on different scales. Min-max normalization maps all scores to [0,1] range:
```
normalized_score = (score - min_score) / (max_score - min_score)
```

This ensures fair combination across methods, preventing one method from dominating due to larger score magnitudes.

Hybrid retrieval typically improves both **recall** (finding relevant documents) and **robustness** (handling diverse query types).

In [12]:
# Hybrid retrieval combining BM25 and semantic embeddings
# This approach leverages both lexical and semantic matching

def normalize_scores(scores):
    """
    Normalize scores to [0, 1] range using min-max normalization.
    Handles edge case where all scores are identical.
    
    Args:
        scores (list): List of numerical scores
    
    Returns:
        list: Normalized scores in [0, 1] range
    """
    if not scores:
        return []
    
    min_score = min(scores)
    max_score = max(scores)
    
    # Handle case where all scores are identical
    if max_score == min_score:
        return [1.0] * len(scores)  # Give all equal high score
    
    # Apply min-max normalization
    normalized = [(score - min_score) / (max_score - min_score) for score in scores]
    return normalized

def hybrid_retrieve(query_text, top_k_lex=15, top_k_sem=15):
    """
    Combine lexical (BM25) and semantic retrieval results.
    Uses union of candidates and normalizes scores for fair comparison.
    
    Args:
        query_text (str): The search query
        top_k_lex (int): Number of lexical results to retrieve
        top_k_sem (int): Number of semantic results to retrieve
    
    Returns:
        pd.DataFrame: Combined results with normalized scores
    """
    # Get BM25 results on original documents (not chunks)
    bm25_results = query_bm25(query_text, top_k=top_k_lex)
    
    # Get semantic results on chunks
    semantic_results = semantic_search(query_text, top_k=top_k_sem)
    
    # Create unified candidate list
    candidates = {}
    
    # Process BM25 results
    bm25_scores = [score for _, score, _ in bm25_results]
    normalized_bm25_scores = normalize_scores(bm25_scores)
    
    for (doc_idx, score, doc_info), norm_score in zip(bm25_results, normalized_bm25_scores):
        doc_id = doc_info['id']
        if doc_id not in candidates:
            candidates[doc_id] = {
                'doc_id': doc_id,
                'title': doc_info['title'],
                'text': doc_info['text'],
                'bm25_score': score,
                'bm25_rank': len(candidates) + 1,
                'bm25_normalized': norm_score,
                'semantic_score': 0,
                'semantic_rank': None,
                'semantic_normalized': 0
            }
    
    # Process semantic results
    semantic_scores = [score for _, score, _ in semantic_results]
    normalized_semantic_scores = normalize_scores(semantic_scores)
    
    for (chunk_idx, score, chunk_info), norm_score in zip(semantic_results, normalized_semantic_scores):
        doc_id = chunk_info['doc_id']
        
        if doc_id not in candidates:
            # Find original document info for new semantic candidates
            orig_doc = corpus_df[corpus_df['id'] == doc_id].iloc[0]
            candidates[doc_id] = {
                'doc_id': doc_id,
                'title': orig_doc['title'],
                'text': orig_doc['text'],
                'bm25_score': 0,
                'bm25_rank': None,
                'bm25_normalized': 0,
                'semantic_score': score,
                'semantic_rank': len([r for r in semantic_results if r[2]['doc_id'] == doc_id]) + 1,
                'semantic_normalized': norm_score
            }
        else:
            # Update existing candidate with semantic info
            # Take best semantic score if multiple chunks from same document
            if score > candidates[doc_id]['semantic_score']:
                candidates[doc_id]['semantic_score'] = score
                candidates[doc_id]['semantic_normalized'] = norm_score
                candidates[doc_id]['semantic_rank'] = len([r for r in semantic_results if r[2]['doc_id'] == doc_id]) + 1
    
    # Convert to DataFrame for easy manipulation
    hybrid_results = pd.DataFrame.from_dict(candidates, orient='index')
    
    return hybrid_results

# Test hybrid retrieval
test_query = "black hole formation from stellar collapse"

print(f"🔍 Testing hybrid retrieval for: '{test_query}'")
hybrid_df = hybrid_retrieve(test_query, top_k_lex=10, top_k_sem=10)

print(f"\n📊 Found {len(hybrid_df)} unique documents from hybrid approach")

# Show method comparison
bm25_only = hybrid_df[hybrid_df['bm25_rank'].notna()].shape[0]
semantic_only = hybrid_df[hybrid_df['semantic_rank'].notna()].shape[0]
both_methods = hybrid_df[(hybrid_df['bm25_rank'].notna()) & (hybrid_df['semantic_rank'].notna())].shape[0]

print(f"\n📈 Method coverage:")
print(f"  BM25 found: {bm25_only} documents")
print(f"  Semantic found: {semantic_only} documents")
print(f"  Both methods found: {both_methods} documents")
print(f"  Union: {len(hybrid_df)} documents")

# Display top hybrid results sorted by combined score
print(f"\n📋 Top 5 candidates for rank fusion:")
print("ID\tTitle\tBM25_norm\tSem_norm\tBM25_rank\tSem_rank")
print("-" * 80)

for idx, row in hybrid_df.head().iterrows():
    bm25_rank = f"{row['bm25_rank']:.0f}" if pd.notna(row['bm25_rank']) else "--"
    sem_rank = f"{row['semantic_rank']:.0f}" if pd.notna(row['semantic_rank']) else "--"
    
    print(f"{row['doc_id']}\t{row['title'][:30]}...\t"
          f"{row['bm25_normalized']:.2f}\t\t{row['semantic_normalized']:.2f}\t\t"
          f"{bm25_rank}\t\t{sem_rank}")

🔍 Testing hybrid retrieval for: 'black hole formation from stellar collapse'

📊 Found 15 unique documents from hybrid approach

📈 Method coverage:
  BM25 found: 10 documents
  Semantic found: 10 documents
  Both methods found: 5 documents
  Union: 15 documents

📋 Top 5 candidates for rank fusion:
ID	Title	BM25_norm	Sem_norm	BM25_rank	Sem_rank
--------------------------------------------------------------------------------
ast_001	Black Holes: Cosmic Vacuum Cle...	1.00		0.82		1		2
ast_005	Stellar Collapse and Black Hol...	0.96		1.00		2		2
ast_003	Exoplanets: Worlds Beyond Our ...	0.56		0.21		3		2
ast_002	The Life Cycle of Stars...	0.43		0.78		4		2
health_003	Nutrition: Fuel for the Human ...	0.30		0.00		5		--


## Rank Fusion: Reciprocal Rank Fusion (RRF)

**Reciprocal Rank Fusion (RRF)** is a simple yet effective method for combining rankings from multiple retrieval systems. It's particularly robust because it relies on ranks rather than raw scores.

### RRF Formula
For each document, RRF computes:
```
RRF_score = Σ (1 / (k + rank_i))
```
where:
- `rank_i` is the document's rank in system i
- `k` is a constant (typically 60) that controls the contribution curve
- The sum is over all systems that retrieved the document

### Why RRF Works Well
1. **Rank-based**: Avoids issues with different score scales
2. **Robust**: Less sensitive to outliers than score-based fusion
3. **Simple**: No parameter tuning beyond choosing k
4. **Proven**: Works well across different retrieval types

### Parameter k
- **k=60** (default): Balanced contribution from all ranks
- **Lower k**: Top ranks dominate more
- **Higher k**: More uniform contribution across ranks

Documents appearing in multiple systems get higher RRF scores, while high-ranking documents in any single system also score well.

In [13]:
# Reciprocal Rank Fusion (RRF) implementation
# RRF is a robust method for combining rankings from different retrieval systems

def rrf_fuse(rankings, k=60):
    """
    Apply Reciprocal Rank Fusion to combine multiple ranking methods.
    
    Args:
        rankings (dict): Dictionary mapping method names to lists of (doc_id, rank) tuples
        k (int): RRF parameter controlling rank contribution curve (default: 60)
    
    Returns:
        list: Tuples of (doc_id, rrf_score, method_details)
    """
    # Collect all unique document IDs
    all_doc_ids = set()
    for method_rankings in rankings.values():
        all_doc_ids.update(doc_id for doc_id, _ in method_rankings)
    
    # Calculate RRF scores for each document
    rrf_scores = {}
    method_details = {}
    
    for doc_id in all_doc_ids:
        total_score = 0
        doc_method_info = {}
        
        # Sum reciprocal ranks across all methods that retrieved this document
        for method_name, method_rankings in rankings.items():
            # Find this document's rank in the current method
            doc_rank = None
            for d_id, rank in method_rankings:
                if d_id == doc_id:
                    doc_rank = rank
                    break
            
            if doc_rank is not None:
                # Calculate reciprocal rank contribution
                contribution = 1 / (k + doc_rank)
                total_score += contribution
                doc_method_info[method_name] = {
                    'rank': doc_rank,
                    'contribution': contribution
                }
            else:
                doc_method_info[method_name] = {
                    'rank': None,
                    'contribution': 0
                }
        
        rrf_scores[doc_id] = total_score
        method_details[doc_id] = doc_method_info
    
    # Sort by RRF score (highest first)
    sorted_results = sorted(
        [(doc_id, score, method_details[doc_id]) 
         for doc_id, score in rrf_scores.items()],
        key=lambda x: x[1],
        reverse=True
    )
    
    return sorted_results

def apply_rrf_to_hybrid(hybrid_df, k=60):
    """
    Apply RRF to hybrid retrieval results.
    
    Args:
        hybrid_df (pd.DataFrame): Hybrid retrieval results
        k (int): RRF parameter
    
    Returns:
        list: RRF fused results
    """
    # Prepare rankings for RRF
    rankings = {}
    
    # BM25 rankings (only for documents that have BM25 results)
    bm25_docs = hybrid_df[hybrid_df['bm25_rank'].notna()]
    if len(bm25_docs) > 0:
        bm25_rankings = [(row['doc_id'], row['bm25_rank']) 
                        for _, row in bm25_docs.iterrows()]
        rankings['BM25'] = bm25_rankings
    
    # Semantic rankings (only for documents that have semantic results)
    semantic_docs = hybrid_df[hybrid_df['semantic_rank'].notna()]
    if len(semantic_docs) > 0:
        semantic_rankings = [(row['doc_id'], row['semantic_rank']) 
                           for _, row in semantic_docs.iterrows()]
        rankings['Semantic'] = semantic_rankings
    
    # Apply RRF
    rrf_results = rrf_fuse(rankings, k=k)
    
    # Enrich results with document information
    enriched_results = []
    for doc_id, rrf_score, method_info in rrf_results:
        doc_row = hybrid_df[hybrid_df['doc_id'] == doc_id].iloc[0]
        enriched_results.append({
            'doc_id': doc_id,
            'title': doc_row['title'],
            'text': doc_row['text'],
            'rrf_score': rrf_score,
            'method_info': method_info,
            'bm25_rank': doc_row['bm25_rank'] if pd.notna(doc_row['bm25_rank']) else None,
            'semantic_rank': doc_row['semantic_rank'] if pd.notna(doc_row['semantic_rank']) else None
        })
    
    return enriched_results

# Test RRF on our hybrid results
test_query = "black hole formation from stellar collapse"
print(f"🔍 Applying RRF to query: '{test_query}'")

# Get hybrid results
hybrid_df = hybrid_retrieve(test_query, top_k_lex=10, top_k_sem=10)

# Apply RRF
rrf_results = apply_rrf_to_hybrid(hybrid_df, k=60)

print(f"\n📊 RRF Results (k=60):")
print("\nTop 10 documents after RRF fusion:")
print("Rank\tDoc ID\tTitle\t\t\tRRF Score\tBM25 Rank\tSem Rank")
print("-" * 85)

for rank, result in enumerate(rrf_results[:10], 1):
    title_short = result['title'][:25] + '...' if len(result['title']) > 25 else result['title']
    bm25_rank = f"{result['bm25_rank']:.0f}" if result['bm25_rank'] is not None else "--"
    sem_rank = f"{result['semantic_rank']:.0f}" if result['semantic_rank'] is not None else "--"
    
    print(f"{rank}\t{result['doc_id']}\t{title_short:<28}\t{result['rrf_score']:.4f}\t\t{bm25_rank}\t\t{sem_rank}")

# Analyze method contributions
print(f"\n📈 Method contribution analysis for top 5 results:")
for i, result in enumerate(rrf_results[:5], 1):
    print(f"\n{i}. [{result['doc_id']}] {result['title']}")
    print(f"   Total RRF score: {result['rrf_score']:.4f}")
    
    for method, info in result['method_info'].items():
        if info['rank'] is not None:
            print(f"   {method}: rank {info['rank']:.0f} → contribution {info['contribution']:.4f}")
        else:
            print(f"   {method}: not found → contribution 0.0000")

# Test different k values to show effect
print(f"\n🔬 Effect of different k values on top result:")
for k_val in [10, 30, 60, 100]:
    rrf_k = apply_rrf_to_hybrid(hybrid_df, k=k_val)
    top_result = rrf_k[0]
    print(f"k={k_val:3d}: [{top_result['doc_id']}] {top_result['title'][:40]}... (RRF: {top_result['rrf_score']:.4f})")

🔍 Applying RRF to query: 'black hole formation from stellar collapse'

📊 RRF Results (k=60):

Top 10 documents after RRF fusion:
Rank	Doc ID	Title			RRF Score	BM25 Rank	Sem Rank
-------------------------------------------------------------------------------------
1	ast_001	Black Holes: Cosmic Vacuu...	0.0325		1		2
2	ast_005	Stellar Collapse and Blac...	0.0323		2		2
3	ast_003	Exoplanets: Worlds Beyond...	0.0320		3		2
4	ast_002	The Life Cycle of Stars     	0.0318		4		2
5	ast_004	The Expanding Universe an...	0.0306		9		2
6	cook_005	Browning and Caramelizati...	0.0161		--		2
7	cook_002	Knife Skills: Foundation ...	0.0161		--		2
8	cook_001	The Maillard Reaction: Sc...	0.0161		--		2
9	hist_001	The Fall of Constantinopl...	0.0161		--		2
10	hist_002	The Printing Press Revolu...	0.0161		--		2

📈 Method contribution analysis for top 5 results:

1. [ast_001] Black Holes: Cosmic Vacuum Cleaners
   Total RRF score: 0.0325
   BM25: rank 1 → contribution 0.0164
   Semantic: rank 2 → contribution 0.01

## Re-ranking with Cross-Encoders

**Cross-encoders** are transformer models that score (query, passage) pairs directly, providing more accurate relevance estimates than individual embeddings. They're the "second stage" in a two-stage retrieval pipeline.

### How Cross-Encoders Work
Unlike bi-encoders (like sentence-transformers) that encode query and document separately, cross-encoders:
1. Concatenate query and passage as input: `[CLS] query [SEP] passage [SEP]`
2. Use full attention across query-passage pairs
3. Output a single relevance score

### Advantages
- **Higher accuracy**: Full attention between query and passage
- **Better ranking**: Specifically trained for relevance scoring
- **Fine-grained scoring**: Can distinguish subtle relevance differences

### Trade-offs
- **Computational cost**: Must process each (query, candidate) pair
- **Latency**: Slower than embedding-based similarity
- **Scale limitations**: Practical only for re-ranking small candidate sets

### Best Practice
Use cross-encoders to re-rank the top-N (typically 10-50) candidates from faster retrieval methods. This gives you both speed and accuracy.

In [ ]:
# Re-ranking with open-source cross-encoder models
# Cross-encoders provide more accurate relevance scoring for final ranking
import time

# Load a lightweight cross-encoder model
# ms-marco-MiniLM is trained on Microsoft's passage ranking dataset
cross_encoder_model_name = "cross-encoder/ms-marco-MiniLM-L-6-v2"
# Alternative models (commented for reference):
# cross_encoder_model_name = "BAAI/bge-reranker-base"          # Higher quality, larger
# cross_encoder_model_name = "cross-encoder/ms-marco-TinyBERT-L-2-v2"  # Faster, smaller

print(f"🤖 Loading cross-encoder: {cross_encoder_model_name}")
cross_encoder = CrossEncoder(cross_encoder_model_name)
print("✅ Cross-encoder loaded successfully")

def rerank_with_cross_encoder(query_text, candidates, top_k=10):
    """
    Re-rank candidates using a cross-encoder model.
    
    Args:
        query_text (str): The search query
        candidates (list): List of candidate documents with text
        top_k (int): Number of top results to return after re-ranking
    
    Returns:
        list: Re-ranked candidates with cross-encoder scores
    """
    if not candidates:
        return []
    
    start_time = time.time()
    
    # Prepare (query, passage) pairs for the cross-encoder
    # Use document title + text for better context
    query_passage_pairs = []
    for candidate in candidates:
        # Combine title and text for richer passage representation
        passage_text = f"{candidate['title']}. {candidate['text']}"
        query_passage_pairs.append([query_text, passage_text])
    
    # Get relevance scores from cross-encoder
    # Scores are logits that can be interpreted as relevance strength
    print(f"🔄 Computing cross-encoder scores for {len(query_passage_pairs)} candidates...")
    relevance_scores = cross_encoder.predict(query_passage_pairs)
    
    # Combine candidates with their cross-encoder scores
    scored_candidates = []
    for candidate, ce_score in zip(candidates, relevance_scores):
        scored_candidate = candidate.copy()
        scored_candidate['cross_encoder_score'] = float(ce_score)
        scored_candidates.append(scored_candidate)
    
    # Sort by cross-encoder score (highest first)
    reranked = sorted(scored_candidates, 
                     key=lambda x: x['cross_encoder_score'], 
                     reverse=True)
    
    elapsed_time = time.time() - start_time
    print(f"⏱️  Cross-encoder re-ranking completed in {elapsed_time:.2f} seconds")
    
    return reranked[:top_k]

# Test cross-encoder re-ranking on RRF results
test_query = "black hole formation from stellar collapse"
print(f"\n🔍 Testing cross-encoder re-ranking for: '{test_query}'")

# Get RRF results as candidates for re-ranking
hybrid_df = hybrid_retrieve(test_query, top_k_lex=15, top_k_sem=15)
rrf_candidates = apply_rrf_to_hybrid(hybrid_df, k=60)

# Take top 20 RRF candidates for re-ranking (manageable size for cross-encoder)
candidates_for_reranking = rrf_candidates[:20]

print(f"\n📊 Re-ranking top {len(candidates_for_reranking)} RRF candidates")

# Apply cross-encoder re-ranking
reranked_results = rerank_with_cross_encoder(
    test_query, 
    candidates_for_reranking, 
    top_k=10
)

# Display comparison: RRF order vs Cross-encoder order
print(f"\n📈 Comparison: RRF vs Cross-Encoder ranking")
print("\nRRF Ranking (before re-ranking):")
print("Rank\tDoc ID\tTitle\t\t\t\tRRF Score")
print("-" * 70)
for i, candidate in enumerate(candidates_for_reranking[:5], 1):
    title_short = candidate['title'][:35] + '...' if len(candidate['title']) > 35 else candidate['title']
    print(f"{i}\t{candidate['doc_id']}\t{title_short:<38}\t{candidate['rrf_score']:.4f}")

print(f"\nCross-Encoder Ranking (after re-ranking):")
print("Rank\tDoc ID\tTitle\t\t\t\tCE Score")
print("-" * 70)
for i, result in enumerate(reranked_results[:5], 1):
    title_short = result['title'][:35] + '...' if len(result['title']) > 35 else result['title']
    print(f"{i}\t{result['doc_id']}\t{title_short:<38}\t{result['cross_encoder_score']:.4f}")

# Show detailed results with snippets
print(f"\n📋 Top 5 results with snippets:")
for i, result in enumerate(reranked_results[:5], 1):
    print(f"\n{i}. [{result['doc_id']}] {result['title']}")
    print(f"   Cross-encoder score: {result['cross_encoder_score']:.4f}")
    print(f"   Original RRF score: {result['rrf_score']:.4f}")
    
    # Show first 200 characters as snippet
    snippet = result['text'][:200] + '...' if len(result['text']) > 200 else result['text']
    print(f"   Snippet: {snippet}")

# Analyze ranking changes
print(f"\n🔄 Ranking changes analysis:")
rrf_order = {cand['doc_id']: i for i, cand in enumerate(candidates_for_reranking)}
ce_order = {result['doc_id']: i for i, result in enumerate(reranked_results)}

rank_changes = []
for doc_id in ce_order.keys():
    if doc_id in rrf_order:
        change = rrf_order[doc_id] - ce_order[doc_id]  # Positive = moved up
        rank_changes.append((doc_id, change))

# Show biggest ranking changes
rank_changes.sort(key=lambda x: abs(x[1]), reverse=True)
print("Biggest ranking changes:")
for doc_id, change in rank_changes[:5]:
    doc_title = next(r['title'] for r in reranked_results if r['doc_id'] == doc_id)
    direction = "↑" if change > 0 else "↓" if change < 0 else "="
    print(f"  [{doc_id}] {direction} {abs(change)} positions: {doc_title[:50]}...")

🤖 Loading cross-encoder: cross-encoder/ms-marco-MiniLM-L-6-v2


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

✅ Cross-encoder loaded successfully

🔍 Testing cross-encoder re-ranking for: 'black hole formation from stellar collapse'

📊 Re-ranking top 20 RRF candidates
🔄 Computing cross-encoder scores for 20 candidates...
⏱️  Cross-encoder re-ranking completed in 0.33 seconds

📈 Comparison: RRF vs Cross-Encoder ranking

RRF Ranking (before re-ranking):
Rank	Doc ID	Title				RRF Score
----------------------------------------------------------------------
1	ast_001	Black Holes: Cosmic Vacuum Cleaners   	0.0325
2	ast_005	Stellar Collapse and Black Hole For...	0.0323
3	ast_003	Exoplanets: Worlds Beyond Our Solar...	0.0320
4	ast_002	The Life Cycle of Stars               	0.0318
5	hist_004	The Industrial Revolution's Impact    	0.0313

Cross-Encoder Ranking (after re-ranking):
Rank	Doc ID	Title				CE Score
----------------------------------------------------------------------
1	ast_005	Stellar Collapse and Black Hole For...	8.8854
2	ast_001	Black Holes: Cosmic Vacuum Cleaners   	4.3416
3	ast_002	The Li

## LLM Generation: Bringing It All Together

The **generation** step combines our retrieved and re-ranked documents with a language model to produce final answers. This is where RAG "augments" the generation with retrieved knowledge.

### Key Components
1. **Context Selection**: Choose top-k documents within token budget
2. **Prompt Engineering**: Structure context and instructions clearly
3. **Citation**: Enable traceability back to source documents
4. **Grounding**: Instruct the model to use only provided context

### Prompt Structure
A well-structured RAG prompt includes:
- **System message**: Instructions for behavior and citation
- **Context section**: Retrieved documents with clear formatting
- **Query**: User's original question
- **Instructions**: Explicit grounding requirements

### Token Budget Management
Language models have context limits. We must:
- Prioritize highest-ranked documents
- Truncate or summarize if needed
- Leave space for the generated response

### Citation Strategy
Include document IDs in the context so the model can reference specific sources. This enables fact-checking and builds user trust.

**Environment handling**: Read API keys from environment variables and provide graceful fallbacks for missing credentials.

In [ ]:
# End-to-end RAG pipeline with OpenAI generation
# This combines all our retrieval components with final answer generation
import os
from openai import OpenAI

def estimate_tokens(text):
    """
    Rough estimation of token count (approximately 4 characters per token).
    This is a simple heuristic; actual tokenization may differ.
    
    Args:
        text (str): Input text
    
    Returns:
        int: Estimated token count
    """
    return len(text) // 4

def select_context_chunks(ranked_results, max_tokens=2000):
    """
    Select top-ranked documents that fit within token budget.
    
    Args:
        ranked_results (list): Ranked documents from retrieval pipeline
        max_tokens (int): Maximum tokens to use for context
    
    Returns:
        list: Selected documents within token budget
    """
    selected_chunks = []
    total_tokens = 0
    
    for result in ranked_results:
        # Estimate tokens for this document (title + text + formatting)
        doc_text = f"[{result['doc_id']}] {result['title']}\n{result['text']}"
        doc_tokens = estimate_tokens(doc_text)
        
        # Check if adding this document would exceed budget
        if total_tokens + doc_tokens <= max_tokens:
            selected_chunks.append(result)
            total_tokens += doc_tokens
        else:
            break
    
    return selected_chunks, total_tokens

def create_rag_prompt(query, context_chunks):
    """
    Create a structured prompt for RAG generation.
    
    Args:
        query (str): User's question
        context_chunks (list): Selected context documents
    
    Returns:
        tuple: (system_message, user_message)
    """
    # System message with clear instructions
    system_message = """You are a helpful assistant that answers questions based on provided context. 

INSTRUCTIONS:
1. Answer the user's question using ONLY the information provided in the context below
2. If you cite information, include the document ID in brackets [doc_id]
3. If the context doesn't contain enough information to answer the question, say so clearly
4. Be accurate and specific - don't make assumptions beyond what's stated in the context
5. Provide a clear, well-structured answer
"""
    
    # Format context documents clearly
    context_text = "\n\nCONTEXT DOCUMENTS:\n\n"
    for i, chunk in enumerate(context_chunks, 1):
        context_text += f"Document {i}: [{chunk['doc_id']}]\n"
        context_text += f"Title: {chunk['title']}\n"
        context_text += f"Content: {chunk['text']}\n\n"
    
    # User message with query and context
    user_message = f"{context_text}\nQUESTION: {query}\n\nPlease provide a comprehensive answer based on the context above."
    
    return system_message, user_message

def answer_query(query_text, max_context_tokens=2000):
    """
    Complete RAG pipeline: retrieve, re-rank, and generate answer.
    
    Args:
        query_text (str): User's question
        max_context_tokens (int): Maximum tokens for context
    
    Returns:
        dict: Complete RAG results including retrieval steps and final answer
    """
    print(f"🔍 Starting RAG pipeline for: '{query_text}'")
    
    # Step 1: Hybrid retrieval (BM25 + Semantic)
    print("📊 Step 1: Hybrid retrieval...")
    hybrid_results = hybrid_retrieve(query_text, top_k_lex=15, top_k_sem=15)
    
    # Step 2: Rank fusion with RRF
    print("🔄 Step 2: Rank fusion (RRF)...")
    rrf_results = apply_rrf_to_hybrid(hybrid_results, k=60)
    
    # Step 3: Cross-encoder re-ranking
    print("🎯 Step 3: Cross-encoder re-ranking...")
    top_candidates = rrf_results[:20]  # Re-rank top 20 candidates
    reranked_results = rerank_with_cross_encoder(query_text, top_candidates, top_k=15)
    
    # Step 4: Context selection within token budget
    print("📝 Step 4: Context selection...")
    selected_context, context_tokens = select_context_chunks(reranked_results, max_context_tokens)
    print(f"   Selected {len(selected_context)} documents using ~{context_tokens} tokens")
    
    # Step 5: Generate answer with OpenAI
    print("🤖 Step 5: Generating answer...")
    
    # Check for OpenAI API key
    openai_api_key = os.getenv('OPENAI_API_KEY')
    if not openai_api_key:
        print("⚠️  OpenAI API key not found in environment variables")
        print("   Set OPENAI_API_KEY environment variable to enable generation")
        return {
            'query': query_text,
            'retrieval_results': len(hybrid_results),
            'rrf_results': len(rrf_results),
            'reranked_results': len(reranked_results),
            'selected_context': selected_context,
            'context_tokens': context_tokens,
            'answer': "[Generation skipped: OpenAI API key not available]",
            'citations': [chunk['doc_id'] for chunk in selected_context]
        }
    
    try:
        # Initialize OpenAI client
        client = OpenAI(api_key=openai_api_key)
        
        # Create RAG prompt
        system_message, user_message = create_rag_prompt(query_text, selected_context)
        
        # Call OpenAI API
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # Fast, cost-effective model
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_message}
            ],
            max_tokens=500,  # Limit response length
            temperature=0.1  # Low temperature for factual responses
        )
        
        answer = response.choices[0].message.content
        
    except Exception as e:
        print(f"❌ Error during generation: {str(e)}")
        answer = f"[Generation failed: {str(e)}]"
    
    # Return comprehensive results
    return {
        'query': query_text,
        'retrieval_results': len(hybrid_results),
        'rrf_results': len(rrf_results),
        'reranked_results': len(reranked_results),
        'selected_context': selected_context,
        'context_tokens': context_tokens,
        'answer': answer,
        'citations': [chunk['doc_id'] for chunk in selected_context]
    }

# Test the complete RAG pipeline
test_queries = [
    "How do black holes form and what happens at the event horizon?",
    "What are Python decorators and how do they modify function behavior?"
]

print("🚀 Testing complete RAG pipeline:\n")
for i, query in enumerate(test_queries, 1):
    print(f"=" * 80)
    print(f"TEST {i}: {query}")
    print(f"=" * 80)
    
    # Run complete RAG pipeline
    rag_result = answer_query(query)
    
    # Display results
    print(f"\n📊 Pipeline Summary:")
    print(f"   Hybrid retrieval: {rag_result['retrieval_results']} candidates")
    print(f"   RRF fusion: {rag_result['rrf_results']} documents")
    print(f"   Re-ranked: {rag_result['reranked_results']} documents")
    print(f"   Context used: {len(rag_result['selected_context'])} documents ({rag_result['context_tokens']} tokens)")
    
    print(f"\n📚 Context Documents:")
    for j, doc in enumerate(rag_result['selected_context'], 1):
        print(f"   {j}. [{doc['doc_id']}] {doc['title']}")
    
    print(f"\n💬 Generated Answer:")
    print(rag_result['answer'])
    print(f"\n🔗 Citations: {', '.join(rag_result['citations'])}")
    print("\n")

## Choosing the Right Approach: Vector Store vs Prompt-Embedded vs Local Files

The choice of knowledge storage and retrieval architecture depends on your specific requirements:

### Vector Stores (e.g., Pinecone, Weaviate, Chroma)
**Best for**: Medium to large corpora, frequent updates, production systems
- **Pros**: Optimized ANN search, metadata filtering, horizontal scaling, real-time updates
- **Cons**: Additional infrastructure, cost, complexity
- **Use when**: >10,000 documents, multiple users, frequent content updates

### Prompt-Embedded Dataset
**Best for**: Very small, static knowledge bases
- **Pros**: Simplest implementation, no retrieval needed, perfect recall
- **Cons**: Limited by context window, expensive tokens, no semantic search
- **Use when**: <10 short documents, completely static content, maximum simplicity

### Local File Embeddings (Our Approach)
**Best for**: Small to medium corpora, single-node applications, development
- **Pros**: No external dependencies, fast development, full control, offline capability
- **Cons**: No horizontal scaling, manual index updates, limited concurrent access
- **Use when**: <100,000 documents, single-node deployment, development/prototyping

### Migration Path
Start with local files for development, then migrate to a vector store when you need:
- More than ~50,000 documents
- Real-time updates
- Multiple concurrent users
- Advanced filtering capabilities

## Glossary of RAG Terms

- **Document**: A single piece of content in your knowledge base (article, page, etc.)
- **Chunk**: A segment of a document, typically 100-500 tokens for better embedding quality
- **Corpus**: The complete collection of documents available for retrieval
- **Index**: Data structure enabling fast search (TF-IDF matrix, embedding vectors, etc.)
- **TF-IDF**: Term Frequency-Inverse Document Frequency; scores terms by frequency vs rarity
- **BM25**: Best Matching 25; probabilistic ranking function improving on TF-IDF
- **Embedding**: Dense vector representation capturing semantic meaning of text
- **Vector Store**: Database optimized for storing and searching high-dimensional vectors
- **ANN**: Approximate Nearest Neighbors; fast similarity search with slight accuracy trade-off
- **FAISS**: Facebook AI Similarity Search; library for efficient similarity search
- **Hybrid Retrieval**: Combining multiple retrieval methods (lexical + semantic)
- **RRF**: Reciprocal Rank Fusion; method for combining rankings from multiple systems
- **Cross-encoder**: Transformer model scoring (query, passage) pairs for re-ranking
- **Top-k**: Retrieving the k highest-scoring results
- **Recall**: Fraction of relevant documents successfully retrieved
- **Precision**: Fraction of retrieved documents that are actually relevant
- **Context Window**: Maximum input length a language model can process
- **Hallucination**: When language models generate factually incorrect information
- **Prompt Template**: Structured format for providing context and instructions to LLMs
- **Grounding**: Ensuring model responses are based on provided evidence rather than training data

In [18]:
# Simple evaluation harness to compare retrieval methods
# This provides quantitative comparison of different approaches

# Define evaluation queries with expected relevant document IDs
# These are hand-crafted based on our synthetic corpus
evaluation_queries_simple = [
    {
        'query': 'black hole formation stellar collapse',
        'relevant_docs': ['ast_001', 'ast_005']  # Black hole docs
    },
    {
        'query': 'python decorators function modification',
        'relevant_docs': ['py_002']  # Decorators doc
    },
    {
        'query': 'exercise cardiovascular heart health',
        'relevant_docs': ['health_002']  # Exercise and cardiovascular health
    },
    {
        'query': 'cooking browning maillard reaction',
        'relevant_docs': ['cook_001', 'cook_005']  # Maillard and browning docs
    },
    {
        'query': 'trade routes silk road medieval',
        'relevant_docs': ['hist_003', 'hist_005']  # Trade route docs
    }
]

evaluation_queries_mixed = [
    # --- EASY (BM25-friendly) ---
    {
        "query": "black hole formation stellar collapse",
        "relevant_docs": ["ast_001", "ast_005"]
    },
    {
        "query": "python decorators function modification",
        "relevant_docs": ["py_002"]
    },
    {
        "query": "exercise cardiovascular heart health",
        "relevant_docs": ["health_002"]
    },
    {
        "query": "cooking browning maillard reaction",
        "relevant_docs": ["cook_001", "cook_005"]
    },
    {
        "query": "trade routes silk road medieval",
        "relevant_docs": ["hist_003", "hist_005"]
    },
    {
        "query": "python list comprehension squares even numbers",
        "relevant_docs": ["py_001", "py_006"]
    },
    {
        "query": "exception handling try except finally cleanup",
        "relevant_docs": ["py_004"]
    },
    {
        "query": "industrial revolution steam engines factories urbanization",
        "relevant_docs": ["hist_004"]
    },

    # --- HARD (BM25-tricky) ---
    {
        "query": "point of no return around a collapsed star",
        "relevant_docs": ["ast_001", "ast_005"]
    },
    {
        "query": "galactic cores hosting monsters millions of suns",
        "relevant_docs": ["ast_001"]
    },
    {
        "query": "runaway expansion powered by the mysterious 68 percent",
        "relevant_docs": ["ast_004"]
    },
    {
        "query": "not caramelization—the browning that needs amino acids",
        "relevant_docs": ["cook_001", "cook_005"]
    },
    {
        "query": "heat moves by touch swirl and radiation in the kitchen",
        "relevant_docs": ["cook_003"]
    },
    {
        "query": "curl your fingers and rock the blade for uniform dice",
        "relevant_docs": ["cook_002"]
    },
    {
        "query": "teach immunity with safe rehearsals of pathogens",
        "relevant_docs": ["health_001"]
    },
    {
        "query": "night work of the brain: memory glue and waste cleanup",
        "relevant_docs": ["health_004"]
    },
    {
        "query": "caravans carried silk and scriptures across Eurasia’s arteries",
        "relevant_docs": ["hist_003", "hist_005"]
    },
    {
        "query": "the @ syntax to wrap behavior without touching the function",
        "relevant_docs": ["py_002"]
    },
    {
        "query": "envrionments like venv and conda to isolate projects",
        "relevant_docs": ["py_003"]
    },
    {
        "query": "base miles plus intervals to raise threshold in endurance",
        "relevant_docs": ["sport_005"]
    },
    {
        "query": "ice baths compression and sleep as recovery levers",
        "relevant_docs": ["sport_003"]
    }
]

evaluation_queries_hard = [
    {
        "query": "point of no return around a collapsed star",
        "relevant_docs": ["ast_001", "ast_005"]  # event horizon + formation
    },
    {
        "query": "galactic nuclei monsters weighing millions of suns",
        "relevant_docs": ["ast_001"]  # supermassive black holes
    },
    {
        "query": "runaway cosmic expansion driven by the 68 percent stuff",
        "relevant_docs": ["ast_004"]  # dark energy fraction + acceleration
    },
    {
        "query": "the 27% invisible scaffolding vs the 5% ordinary atoms",
        "relevant_docs": ["ast_004"]  # dark matter vs baryonic matter
    },
    {
        "query": "worlds found by watching stars blink when a planet passes",
        "relevant_docs": ["ast_003"]  # transit photometry + Kepler
    },
    {
        "query": "stellar infancy: gas-dust nurseries before the long main act",
        "relevant_docs": ["ast_002"]  # protostar → main sequence
    },
    {
        "query": "massive suns die loudly; small ones cool quietly—explain",
        "relevant_docs": ["ast_002", "ast_005"]  # supernovae vs white dwarfs; collapse
    },
    {
        "query": "not caramelization—name the browning with amino acids",
        "relevant_docs": ["cook_001", "cook_005"]  # Maillard vs caramelization
    },
    {
        "query": "golden crust engineering: heat too low vs too high outcomes",
        "relevant_docs": ["cook_001"]  # temp control for Maillard
    },
    {
        "query": "heat moves by touching, swirling, and radiant glow in kitchens",
        "relevant_docs": ["cook_003"]  # conduction/convection/radiation
    },
    {
        "query": "sour cabbage craft needs salt, temp, and pH discipline",
        "relevant_docs": ["cook_004"]  # lactic fermentation controls
    },
    {
        "query": "keep your digits curled; rock the blade for uniform cubes",
        "relevant_docs": ["cook_002"]  # knife skills safety + consistency
    },
    {
        "query": "the @ sign trick: wrap a function without editing its body",
        "relevant_docs": ["py_002"]  # decorators
    },
    {
        "query": "separate Python worlds so dependencies don’t brawl",
        "relevant_docs": ["py_003"]  # virtual environments
    },
    {
        "query": "catch problems gracefully; always clean up afterward",
        "relevant_docs": ["py_004"]  # try/except/finally
    },
    {
        "query": "iterate without hoarding memory—yield your way through data",
        "relevant_docs": ["py_005"]  # generators
    },
    {
        "query": "double each item, or map then filter for clarity",
        "relevant_docs": ["py_006", "py_001"]  # list processing + list comprehensions
    },
    {
        "query": "AHA weekly target: minutes that lift HDL and tame BP",
        "relevant_docs": ["health_002"]  # 150 minutes + HDL/BP effects
    },
    {
        "query": "teach the body’s defenders with safe rehearsals of pathogens",
        "relevant_docs": ["health_001"]  # vaccines train immune system
    },
    {
        "query": "night shift for the brain: waste clearance and memory glue",
        "relevant_docs": ["health_004"]  # sleep functions
    },
    {
        "query": "mind mechanics of competition: imagery, goals, staying present",
        "relevant_docs": ["sport_002"]  # sports psychology tools
    },
    {
        "query": "ice tubs, squeeze sleeves, and lights-out: the recovery toolbox",
        "relevant_docs": ["sport_003"]  # recovery modalities + sleep
    },
    {
        "query": "add weight over time and favor big multi-joint moves",
        "relevant_docs": ["sport_004"]  # progressive overload + compounds
    },
    {
        "query": "train the engine: base miles plus intervals to raise the threshold",
        "relevant_docs": ["sport_005"]  # endurance + lactate threshold
    },
    {
        "query": "fifty-three days, giant bombards, ancient walls undone",
        "relevant_docs": ["hist_001"]  # Constantinople siege details
    },
    {
        "query": "movable type: the idea copier that turbocharged reform and science",
        "relevant_docs": ["hist_002"]  # printing press impacts
    },
    {
        "query": "caravans of silk and scripture crossed Eurasia’s arteries",
        "relevant_docs": ["hist_003", "hist_005"]  # Silk Road commerce+culture
    },
    {
        "query": "smokestacks, steam, and cities swelling—name the shift",
        "relevant_docs": ["hist_004"]  # Industrial Revolution
    },
    {
        "query": "dekorators that time calls—higher-order wrappers with @",
        "relevant_docs": ["py_002"]  # misspelling + synonyms
    },
    {
        "query": "envrionments: venv/conda/pipenv to sandbox projects",
        "relevant_docs": ["py_003"]  # misspelling + tool names
    }
]

def calculate_hit_at_k(retrieved_doc_ids, relevant_doc_ids, k):
    """
    Calculate Hit@K: whether at least one relevant document appears in top-k results.
    
    Args:
        retrieved_doc_ids (list): List of retrieved document IDs in rank order
        relevant_doc_ids (list): List of relevant document IDs
        k (int): Number of top results to consider
    
    Returns:
        float: 1.0 if hit, 0.0 if miss
    """
    top_k_retrieved = set(retrieved_doc_ids[:k])
    relevant_set = set(relevant_doc_ids)
    
    # Hit if intersection is non-empty
    return 1.0 if top_k_retrieved.intersection(relevant_set) else 0.0

def evaluate_retrieval_method(method_name, retrieval_function, eval_queries, k_values=[5, 10]):
    """
    Evaluate a retrieval method on multiple queries.
    
    Args:
        method_name (str): Name of the retrieval method
        retrieval_function (callable): Function that takes query and returns ranked results
        eval_queries (list): List of evaluation query dictionaries
        k_values (list): List of k values for Hit@K calculation
    
    Returns:
        dict: Evaluation results
    """
    results = {f'hit_at_{k}': [] for k in k_values}
    
    for eval_item in eval_queries:
        query = eval_item['query']
        relevant_docs = eval_item['relevant_docs']
        
        # Get retrieval results
        retrieved_results = retrieval_function(query)
        
        # Extract document IDs from results
        if method_name == 'Semantic':
            # For semantic search, extract doc_id from chunk info
            retrieved_doc_ids = [result[2]['doc_id'] for result in retrieved_results]
        else:
            # For TF-IDF and BM25, extract id from doc info
            retrieved_doc_ids = [result[2]['id'] for result in retrieved_results]
        
        # Calculate Hit@K for each k value
        for k in k_values:
            hit = calculate_hit_at_k(retrieved_doc_ids, relevant_docs, k)
            results[f'hit_at_{k}'].append(hit)
    
    # Calculate average Hit@K scores
    avg_results = {}
    for k in k_values:
        avg_results[f'hit_at_{k}'] = np.mean(results[f'hit_at_{k}'])
    
    return avg_results

def evaluate_hybrid_method(method_name, eval_queries, k_values=[5, 10]):
    """
    Evaluate hybrid methods (RRF, Cross-encoder) on multiple queries.
    """
    results = {f'hit_at_{k}': [] for k in k_values}
    
    for eval_item in eval_queries:
        query = eval_item['query']
        relevant_docs = eval_item['relevant_docs']
        
        if method_name == 'RRF':
            # Get RRF results
            hybrid_df = hybrid_retrieve(query, top_k_lex=15, top_k_sem=15)
            rrf_results = apply_rrf_to_hybrid(hybrid_df, k=60)
            retrieved_doc_ids = [result['doc_id'] for result in rrf_results]
        
        elif method_name == 'Cross-encoder':
            # Get cross-encoder re-ranked results
            hybrid_df = hybrid_retrieve(query, top_k_lex=15, top_k_sem=15)
            rrf_results = apply_rrf_to_hybrid(hybrid_df, k=60)
            reranked_results = rerank_with_cross_encoder(query, rrf_results[:20], top_k=15)
            retrieved_doc_ids = [result['doc_id'] for result in reranked_results]
        
        # Calculate Hit@K for each k value
        for k in k_values:
            hit = calculate_hit_at_k(retrieved_doc_ids, relevant_docs, k)
            results[f'hit_at_{k}'].append(hit)
    
    # Calculate average Hit@K scores
    avg_results = {}
    for k in k_values:
        avg_results[f'hit_at_{k}'] = np.mean(results[f'hit_at_{k}'])
    
    return avg_results

# Run evaluation on all methods
print("🧪 Running retrieval method evaluation...\n")

# Prepare retrieval functions
retrieval_methods = {
    'TF-IDF': lambda q: query_tfidf(q, top_k=10),
    'BM25': lambda q: query_bm25(q, top_k=10),
    'Semantic': lambda q: semantic_search(q, top_k=10)
}

# Evaluate individual methods
evaluation_results = {}
evaluation_queries = evaluation_queries_mixed  # Use hard queries for robust evaluation
for method_name, retrieval_func in retrieval_methods.items():
    print(f"📊 Evaluating {method_name}...")
    results = evaluate_retrieval_method(method_name, retrieval_func, evaluation_queries)
    evaluation_results[method_name] = results

# Evaluate hybrid methods (these require special handling)
print(f"📊 Evaluating RRF...")
evaluation_results['RRF'] = evaluate_hybrid_method('RRF', evaluation_queries)

print(f"📊 Evaluating Cross-encoder...")
evaluation_results['Cross-encoder'] = evaluate_hybrid_method('Cross-encoder', evaluation_queries)

# Display results in a comparison table
print(f"\n📈 Retrieval Method Comparison (Hit@K scores):")
print(f"{'Method':<15} {'Hit@5':<8} {'Hit@10':<8}")
print("-" * 35)

for method_name, results in evaluation_results.items():
    hit_at_5 = results['hit_at_5']
    hit_at_10 = results['hit_at_10']
    print(f"{method_name:<15} {hit_at_5:<8.2f} {hit_at_10:<8.2f}")

# Analysis
print(f"\n📊 Analysis:")
best_hit5 = max(evaluation_results.items(), key=lambda x: x[1]['hit_at_5'])
best_hit10 = max(evaluation_results.items(), key=lambda x: x[1]['hit_at_10'])

print(f"🏆 Best Hit@5: {best_hit5[0]} ({best_hit5[1]['hit_at_5']:.2f})")
print(f"🏆 Best Hit@10: {best_hit10[0]} ({best_hit10[1]['hit_at_10']:.2f})")

# Show method improvements
baseline_hit5 = evaluation_results['TF-IDF']['hit_at_5']
print(f"\n📈 Improvements over TF-IDF baseline:")
for method_name, results in evaluation_results.items():
    if method_name != 'TF-IDF':
        improvement = results['hit_at_5'] - baseline_hit5
        print(f"   {method_name}: {improvement:+.2f} Hit@5")

print("\n✅ Evaluation complete!")

🧪 Running retrieval method evaluation...

📊 Evaluating TF-IDF...
📊 Evaluating BM25...
📊 Evaluating Semantic...
📊 Evaluating RRF...
📊 Evaluating Cross-encoder...
🔄 Computing cross-encoder scores for 20 candidates...
⏱️  Cross-encoder re-ranking completed in 0.06 seconds
🔄 Computing cross-encoder scores for 20 candidates...
⏱️  Cross-encoder re-ranking completed in 0.03 seconds
🔄 Computing cross-encoder scores for 20 candidates...
⏱️  Cross-encoder re-ranking completed in 0.03 seconds
🔄 Computing cross-encoder scores for 19 candidates...
⏱️  Cross-encoder re-ranking completed in 0.04 seconds
🔄 Computing cross-encoder scores for 20 candidates...
⏱️  Cross-encoder re-ranking completed in 0.03 seconds
🔄 Computing cross-encoder scores for 20 candidates...
⏱️  Cross-encoder re-ranking completed in 0.04 seconds
🔄 Computing cross-encoder scores for 20 candidates...
⏱️  Cross-encoder re-ranking completed in 0.04 seconds
🔄 Computing cross-encoder scores for 20 candidates...
⏱️  Cross-encoder re-r

## Enhanced Evaluation Framework with MRR and NDCG

While Hit@K provides a basic measure of retrieval success, more sophisticated metrics offer deeper insights:

### Mean Reciprocal Rank (MRR)
MRR measures how high the first relevant document appears in the ranking. It gives more credit to systems that place relevant documents at the top.

**Formula**: MRR = (1/|Q|) × Σ(1/rank_i) where rank_i is the position of the first relevant document for query i

### Normalized Discounted Cumulative Gain (NDCG)
NDCG accounts for the position of all relevant documents and allows for graded relevance (not just binary relevant/irrelevant).

**Benefits over Hit@K**:
- More sensitive to ranking quality
- Accounts for position of all relevant documents  
- Provides finer-grained performance assessment
- Better for comparing subtle differences between methods

In [20]:
import math
from typing import List, Dict, Set

def calculate_mrr(retrieved_doc_ids: List[str], relevant_doc_ids: List[str]) -> float:
    """
    Calculate Mean Reciprocal Rank for a single query.
    
    MRR measures the quality of a ranking by looking at the position of the first relevant document.
    Higher scores indicate that relevant documents appear earlier in the ranking.
    
    Args:
        retrieved_doc_ids: List of retrieved document IDs in rank order
        relevant_doc_ids: List of known relevant document IDs
    
    Returns:
        float: MRR score (1/rank of first relevant doc, or 0 if no relevant docs found)
    
    Example:
        retrieved = ['doc1', 'doc2', 'doc3', 'doc4', 'doc5']
        relevant = ['doc3', 'doc6']
        MRR = 1/3 = 0.333 (first relevant doc 'doc3' is at position 3)
    """
    relevant_set = set(relevant_doc_ids)
    
    for rank, doc_id in enumerate(retrieved_doc_ids, 1):
        if doc_id in relevant_set:
            return 1.0 / rank
    
    return 0.0  # No relevant documents found


def calculate_dcg_at_k(retrieved_doc_ids: List[str], relevant_doc_ids: List[str], 
                       relevance_scores: Dict[str, float], k: int) -> float:
    """
    Calculate Discounted Cumulative Gain at position k.
    
    DCG measures the usefulness of documents based on their position in the ranking,
    with higher positions having exponentially more impact.
    
    Args:
        retrieved_doc_ids: List of retrieved document IDs in rank order
        relevant_doc_ids: List of known relevant document IDs  
        relevance_scores: Dict mapping doc_id to relevance score (0-3 scale typically)
        k: Calculate DCG for top-k results
    
    Returns:
        float: DCG@k score
    """
    dcg = 0.0
    
    for i, doc_id in enumerate(retrieved_doc_ids[:k]):
        if doc_id in relevance_scores:
            relevance = relevance_scores[doc_id]
            # DCG formula: rel_i / log2(i + 2) where i is 0-indexed position
            dcg += relevance / math.log2(i + 2)
    
    return dcg


def calculate_ndcg_at_k(retrieved_doc_ids: List[str], relevant_doc_ids: List[str],
                        relevance_scores: Dict[str, float], k: int) -> float:
    """
    Calculate Normalized Discounted Cumulative Gain at position k.
    
    NDCG normalizes DCG by the ideal DCG (IDCG) to get a score between 0 and 1.
    This allows fair comparison between queries with different numbers of relevant documents.
    
    Args:
        retrieved_doc_ids: List of retrieved document IDs in rank order
        relevant_doc_ids: List of known relevant document IDs
        relevance_scores: Dict mapping doc_id to relevance score
        k: Calculate NDCG for top-k results
    
    Returns:
        float: NDCG@k score (0-1, where 1 is perfect ranking)
    """
    # Calculate actual DCG
    dcg = calculate_dcg_at_k(retrieved_doc_ids, relevant_doc_ids, relevance_scores, k)
    
    # Calculate Ideal DCG (IDCG) - what we'd get with perfect ranking
    # Sort relevant docs by relevance score in descending order
    ideal_ranking = sorted(relevance_scores.keys(), 
                          key=lambda x: relevance_scores[x], reverse=True)
    idcg = calculate_dcg_at_k(ideal_ranking, relevant_doc_ids, relevance_scores, k)
    
    # NDCG = DCG / IDCG (avoid division by zero)
    return dcg / idcg if idcg > 0 else 0.0


def calculate_precision_at_k(retrieved_doc_ids: List[str], relevant_doc_ids: List[str], k: int) -> float:
    """
    Calculate Precision@K: fraction of retrieved documents that are relevant.
    
    Args:
        retrieved_doc_ids: List of retrieved document IDs in rank order
        relevant_doc_ids: List of known relevant document IDs
        k: Number of top results to consider
    
    Returns:
        float: Precision@k score (0-1)
    """
    if k == 0:
        return 0.0
        
    top_k_retrieved = set(retrieved_doc_ids[:k])
    relevant_set = set(relevant_doc_ids)
    
    relevant_retrieved = top_k_retrieved.intersection(relevant_set)
    return len(relevant_retrieved) / k


def calculate_recall_at_k(retrieved_doc_ids: List[str], relevant_doc_ids: List[str], k: int) -> float:
    """
    Calculate Recall@K: fraction of relevant documents that were retrieved.
    
    Args:
        retrieved_doc_ids: List of retrieved document IDs in rank order  
        relevant_doc_ids: List of known relevant document IDs
        k: Number of top results to consider
    
    Returns:
        float: Recall@k score (0-1)
    """
    if not relevant_doc_ids:
        return 0.0
        
    top_k_retrieved = set(retrieved_doc_ids[:k])
    relevant_set = set(relevant_doc_ids)
    
    relevant_retrieved = top_k_retrieved.intersection(relevant_set)
    return len(relevant_retrieved) / len(relevant_set)


def calculate_f1_at_k(retrieved_doc_ids: List[str], relevant_doc_ids: List[str], k: int) -> float:
    """
    Calculate F1@K: harmonic mean of Precision@K and Recall@K.
    
    F1 provides a single score that balances precision and recall.
    
    Args:
        retrieved_doc_ids: List of retrieved document IDs in rank order
        relevant_doc_ids: List of known relevant document IDs  
        k: Number of top results to consider
    
    Returns:
        float: F1@k score (0-1)
    """
    precision = calculate_precision_at_k(retrieved_doc_ids, relevant_doc_ids, k)
    recall = calculate_recall_at_k(retrieved_doc_ids, relevant_doc_ids, k)
    
    if precision + recall == 0:
        return 0.0
    
    return 2 * (precision * recall) / (precision + recall)


# Create enhanced evaluation queries with graded relevance scores
# For this demo, we'll use a simple binary relevance (relevant=1, not relevant=0)
# In practice, you might have 3-point or 4-point relevance scales

def create_relevance_scores(eval_queries: List[Dict]) -> Dict[str, Dict[str, float]]:
    """
    Create relevance score mappings for evaluation queries.
    
    For simplicity, we use binary relevance: relevant docs get score 1.0, others get 0.0
    In production, you might have multi-level relevance (0=irrelevant, 1=somewhat, 2=relevant, 3=highly relevant)
    """
    relevance_mapping = {}
    
    for i, query_data in enumerate(eval_queries):
        query_id = f"query_{i}"
        relevance_scores = {}
        
        # All relevant docs get score 1.0, irrelevant docs get 0.0
        for doc_id in query_data['relevant_docs']:
            relevance_scores[doc_id] = 1.0
            
        relevance_mapping[query_id] = relevance_scores
    
    return relevance_mapping

# Test the new metrics with a simple example
print("🧪 Testing enhanced evaluation metrics with examples:\n")

# Example 1: Perfect ranking
retrieved_1 = ['doc_a', 'doc_b', 'doc_c', 'doc_d', 'doc_e']
relevant_1 = ['doc_a', 'doc_b']
relevance_1 = {'doc_a': 1.0, 'doc_b': 1.0}

print("📊 Example 1 - Perfect Ranking:")
print(f"   Retrieved: {retrieved_1[:3]}...")
print(f"   Relevant: {relevant_1}")
print(f"   MRR: {calculate_mrr(retrieved_1, relevant_1):.3f}")
print(f"   NDCG@5: {calculate_ndcg_at_k(retrieved_1, relevant_1, relevance_1, 5):.3f}")
print(f"   Precision@5: {calculate_precision_at_k(retrieved_1, relevant_1, 5):.3f}")
print(f"   Recall@5: {calculate_recall_at_k(retrieved_1, relevant_1, 5):.3f}")
print(f"   F1@5: {calculate_f1_at_k(retrieved_1, relevant_1, 5):.3f}")

# Example 2: Poor ranking  
retrieved_2 = ['doc_x', 'doc_y', 'doc_z', 'doc_a', 'doc_b']
relevant_2 = ['doc_a', 'doc_b']
relevance_2 = {'doc_a': 1.0, 'doc_b': 1.0}

print(f"\n📊 Example 2 - Poor Ranking (relevant docs at positions 4,5):")
print(f"   Retrieved: {retrieved_2}")
print(f"   Relevant: {relevant_2}")
print(f"   MRR: {calculate_mrr(retrieved_2, relevant_2):.3f}")
print(f"   NDCG@5: {calculate_ndcg_at_k(retrieved_2, relevant_2, relevance_2, 5):.3f}")
print(f"   Precision@5: {calculate_precision_at_k(retrieved_2, relevant_2, 5):.3f}")
print(f"   Recall@5: {calculate_recall_at_k(retrieved_2, relevant_2, 5):.3f}")
print(f"   F1@5: {calculate_f1_at_k(retrieved_2, relevant_2, 5):.3f}")

print("\n💡 Key Insights:")
print("   • MRR heavily penalizes when first relevant doc is ranked low")
print("   • NDCG accounts for position of ALL relevant documents")  
print("   • Precision@K = relevant_retrieved / k_retrieved")
print("   • Recall@K = relevant_retrieved / total_relevant")
print("   • F1@K balances precision and recall")

🧪 Testing enhanced evaluation metrics with examples:

📊 Example 1 - Perfect Ranking:
   Retrieved: ['doc_a', 'doc_b', 'doc_c']...
   Relevant: ['doc_a', 'doc_b']
   MRR: 1.000
   NDCG@5: 1.000
   Precision@5: 0.400
   Recall@5: 1.000
   F1@5: 0.571

📊 Example 2 - Poor Ranking (relevant docs at positions 4,5):
   Retrieved: ['doc_x', 'doc_y', 'doc_z', 'doc_a', 'doc_b']
   Relevant: ['doc_a', 'doc_b']
   MRR: 0.250
   NDCG@5: 0.501
   Precision@5: 0.400
   Recall@5: 1.000
   F1@5: 0.571

💡 Key Insights:
   • MRR heavily penalizes when first relevant doc is ranked low
   • NDCG accounts for position of ALL relevant documents
   • Precision@K = relevant_retrieved / k_retrieved
   • Recall@K = relevant_retrieved / total_relevant
   • F1@K balances precision and recall


In [21]:
def enhanced_evaluate_retrieval_method(method_name: str, retrieval_function, 
                                      eval_queries: List[Dict], k_values: List[int] = [5, 10]) -> Dict:
    """
    Enhanced evaluation using multiple metrics: Hit@K, MRR, NDCG, Precision, Recall, F1.
    
    Args:
        method_name: Name of the retrieval method
        retrieval_function: Function that takes query and returns ranked results
        eval_queries: List of evaluation query dictionaries
        k_values: List of k values for evaluation
    
    Returns:
        Dict containing averaged scores for all metrics
    """
    metrics = {
        'mrr': [],
        **{f'hit_at_{k}': [] for k in k_values},
        **{f'ndcg_at_{k}': [] for k in k_values},
        **{f'precision_at_{k}': [] for k in k_values},
        **{f'recall_at_{k}': [] for k in k_values},
        **{f'f1_at_{k}': [] for k in k_values}
    }
    
    # Create relevance scores for NDCG calculation
    relevance_mapping = create_relevance_scores(eval_queries)
    
    for i, eval_item in enumerate(eval_queries):
        query = eval_item['query']
        relevant_docs = eval_item['relevant_docs']
        query_id = f"query_{i}"
        relevance_scores = relevance_mapping[query_id]
        
        # Get retrieval results
        retrieved_results = retrieval_function(query)
        
        # Extract document IDs from results (handle different return formats)
        if method_name == 'Semantic':
            # For semantic search, extract doc_id from chunk info
            retrieved_doc_ids = [result[2]['doc_id'] for result in retrieved_results]
        else:
            # For TF-IDF and BM25, extract id from doc info
            retrieved_doc_ids = [result[2]['id'] for result in retrieved_results]
        
        # Calculate MRR (only needs to be calculated once per query)
        mrr = calculate_mrr(retrieved_doc_ids, relevant_docs)
        metrics['mrr'].append(mrr)
        
        # Calculate metrics for each k value
        for k in k_values:
            # Hit@K
            hit = calculate_hit_at_k(retrieved_doc_ids, relevant_docs, k)
            metrics[f'hit_at_{k}'].append(hit)
            
            # NDCG@K
            ndcg = calculate_ndcg_at_k(retrieved_doc_ids, relevant_docs, relevance_scores, k)
            metrics[f'ndcg_at_{k}'].append(ndcg)
            
            # Precision@K
            precision = calculate_precision_at_k(retrieved_doc_ids, relevant_docs, k)
            metrics[f'precision_at_{k}'].append(precision)
            
            # Recall@K
            recall = calculate_recall_at_k(retrieved_doc_ids, relevant_docs, k)
            metrics[f'recall_at_{k}'].append(recall)
            
            # F1@K  
            f1 = calculate_f1_at_k(retrieved_doc_ids, relevant_docs, k)
            metrics[f'f1_at_{k}'].append(f1)
    
    # Calculate averages
    avg_metrics = {}
    for metric_name, values in metrics.items():
        avg_metrics[metric_name] = np.mean(values)
    
    return avg_metrics


def enhanced_evaluate_hybrid_method(method_name: str, eval_queries: List[Dict], 
                                   k_values: List[int] = [5, 10]) -> Dict:
    """
    Enhanced evaluation for hybrid methods (RRF, Cross-encoder) using multiple metrics.
    """
    metrics = {
        'mrr': [],
        **{f'hit_at_{k}': [] for k in k_values},
        **{f'ndcg_at_{k}': [] for k in k_values}, 
        **{f'precision_at_{k}': [] for k in k_values},
        **{f'recall_at_{k}': [] for k in k_values},
        **{f'f1_at_{k}': [] for k in k_values}
    }
    
    # Create relevance scores
    relevance_mapping = create_relevance_scores(eval_queries)
    
    for i, eval_item in enumerate(eval_queries):
        query = eval_item['query']
        relevant_docs = eval_item['relevant_docs']
        query_id = f"query_{i}"
        relevance_scores = relevance_mapping[query_id]
        
        # Get method-specific results
        if method_name == 'RRF':
            hybrid_df = hybrid_retrieve(query, top_k_lex=15, top_k_sem=15)
            rrf_results = apply_rrf_to_hybrid(hybrid_df, k=60)
            retrieved_doc_ids = [result['doc_id'] for result in rrf_results]
        
        elif method_name == 'Cross-encoder':
            hybrid_df = hybrid_retrieve(query, top_k_lex=15, top_k_sem=15)
            rrf_results = apply_rrf_to_hybrid(hybrid_df, k=60)
            reranked_results = rerank_with_cross_encoder(query, rrf_results[:20], top_k=15)
            retrieved_doc_ids = [result['doc_id'] for result in reranked_results]
        
        # Calculate MRR
        mrr = calculate_mrr(retrieved_doc_ids, relevant_docs)
        metrics['mrr'].append(mrr)
        
        # Calculate metrics for each k value
        for k in k_values:
            hit = calculate_hit_at_k(retrieved_doc_ids, relevant_docs, k)
            metrics[f'hit_at_{k}'].append(hit)
            
            ndcg = calculate_ndcg_at_k(retrieved_doc_ids, relevant_docs, relevance_scores, k)
            metrics[f'ndcg_at_{k}'].append(ndcg)
            
            precision = calculate_precision_at_k(retrieved_doc_ids, relevant_docs, k)
            metrics[f'precision_at_{k}'].append(precision)
            
            recall = calculate_recall_at_k(retrieved_doc_ids, relevant_docs, k)
            metrics[f'recall_at_{k}'].append(recall)
            
            f1 = calculate_f1_at_k(retrieved_doc_ids, relevant_docs, k)
            metrics[f'f1_at_{k}'].append(f1)
    
    # Calculate averages
    avg_metrics = {}
    for metric_name, values in metrics.items():
        avg_metrics[metric_name] = np.mean(values)
    
    return avg_metrics

# Run enhanced evaluation on all methods
print("🚀 Running enhanced retrieval evaluation with multiple metrics...\n")

# Use mixed queries for comprehensive evaluation
evaluation_queries = evaluation_queries_mixed[:10]  # Use subset for demo (faster execution)

# Evaluate individual methods
enhanced_results = {}
retrieval_methods = {
    'TF-IDF': lambda q: query_tfidf(q, top_k=10),
    'BM25': lambda q: query_bm25(q, top_k=10),
    'Semantic': lambda q: semantic_search(q, top_k=10)
}

for method_name, retrieval_func in retrieval_methods.items():
    print(f"📊 Evaluating {method_name} with enhanced metrics...")
    results = enhanced_evaluate_retrieval_method(method_name, retrieval_func, evaluation_queries)
    enhanced_results[method_name] = results

# Evaluate hybrid methods
print(f"📊 Evaluating RRF with enhanced metrics...")
enhanced_results['RRF'] = enhanced_evaluate_hybrid_method('RRF', evaluation_queries)

print(f"📊 Evaluating Cross-encoder with enhanced metrics...")
enhanced_results['Cross-encoder'] = enhanced_evaluate_hybrid_method('Cross-encoder', evaluation_queries)

# Display comprehensive results table
print(f"\n📈 Comprehensive Retrieval Evaluation Results:")
print(f"{'Method':<15} {'MRR':<6} {'Hit@5':<6} {'Hit@10':<7} {'NDCG@5':<7} {'NDCG@10':<8} {'P@5':<6} {'R@5':<6} {'F1@5':<6}")
print("-" * 80)

for method_name, results in enhanced_results.items():
    print(f"{method_name:<15} "
          f"{results['mrr']:<6.3f} "
          f"{results['hit_at_5']:<6.3f} "
          f"{results['hit_at_10']:<7.3f} "
          f"{results['ndcg_at_5']:<7.3f} "
          f"{results['ndcg_at_10']:<8.3f} "
          f"{results['precision_at_5']:<6.3f} "
          f"{results['recall_at_5']:<6.3f} "
          f"{results['f1_at_5']:<6.3f}")

# Enhanced analysis
print(f"\n🏆 Best Performers by Metric:")
best_mrr = max(enhanced_results.items(), key=lambda x: x[1]['mrr'])
best_ndcg5 = max(enhanced_results.items(), key=lambda x: x[1]['ndcg_at_5'])
best_f1_5 = max(enhanced_results.items(), key=lambda x: x[1]['f1_at_5'])

print(f"   Best MRR: {best_mrr[0]} ({best_mrr[1]['mrr']:.3f})")
print(f"   Best NDCG@5: {best_ndcg5[0]} ({best_ndcg5[1]['ndcg_at_5']:.3f})")
print(f"   Best F1@5: {best_f1_5[0]} ({best_f1_5[1]['f1_at_5']:.3f})")

# Statistical significance insights
print(f"\n📊 Metric Insights:")
print(f"   • MRR focuses on rank of first relevant document")
print(f"   • NDCG considers position of all relevant documents")
print(f"   • Hit@K is binary (found/not found)")
print(f"   • Precision@K shows relevance ratio in top-K")
print(f"   • Recall@K shows coverage of relevant documents")
print(f"   • F1@K balances precision and recall")

# Show improvement analysis
baseline_method = 'TF-IDF'
baseline_mrr = enhanced_results[baseline_method]['mrr']
baseline_ndcg5 = enhanced_results[baseline_method]['ndcg_at_5']

print(f"\n📈 Improvements over {baseline_method} baseline:")
for method_name, results in enhanced_results.items():
    if method_name != baseline_method:
        mrr_improvement = results['mrr'] - baseline_mrr
        ndcg_improvement = results['ndcg_at_5'] - baseline_ndcg5
        print(f"   {method_name}: MRR {mrr_improvement:+.3f}, NDCG@5 {ndcg_improvement:+.3f}")

print("\n✅ Enhanced evaluation complete!")

🚀 Running enhanced retrieval evaluation with multiple metrics...

📊 Evaluating TF-IDF with enhanced metrics...
📊 Evaluating BM25 with enhanced metrics...
📊 Evaluating Semantic with enhanced metrics...
📊 Evaluating RRF with enhanced metrics...
📊 Evaluating Cross-encoder with enhanced metrics...
🔄 Computing cross-encoder scores for 20 candidates...
⏱️  Cross-encoder re-ranking completed in 0.04 seconds
🔄 Computing cross-encoder scores for 20 candidates...
⏱️  Cross-encoder re-ranking completed in 0.03 seconds
🔄 Computing cross-encoder scores for 20 candidates...
⏱️  Cross-encoder re-ranking completed in 0.04 seconds
🔄 Computing cross-encoder scores for 19 candidates...
⏱️  Cross-encoder re-ranking completed in 0.03 seconds
🔄 Computing cross-encoder scores for 20 candidates...
⏱️  Cross-encoder re-ranking completed in 0.04 seconds
🔄 Computing cross-encoder scores for 20 candidates...
⏱️  Cross-encoder re-ranking completed in 0.04 seconds
🔄 Computing cross-encoder scores for 20 candidates.

## Interpreting Enhanced Evaluation Metrics

### When to Use Each Metric

**Mean Reciprocal Rank (MRR)**
- **Best for**: Systems where finding the first relevant document quickly is critical
- **Example**: Question answering where users need one good answer
- **Interpretation**: MRR=0.5 means on average, the first relevant document is at position 2

**Normalized Discounted Cumulative Gain (NDCG)**  
- **Best for**: Systems where ranking quality of all results matters
- **Example**: Search engines where users browse multiple results
- **Interpretation**: NDCG@5=0.8 means the ranking achieves 80% of the ideal score

**Hit@K**
- **Best for**: Simple binary assessment of retrieval success
- **Example**: Basic "did we find anything useful?" evaluation
- **Interpretation**: Hit@5=0.7 means 70% of queries had at least one relevant doc in top-5

**Precision@K**
- **Best for**: Systems where result quality (low false positives) is crucial
- **Example**: Medical diagnosis support where wrong results are dangerous
- **Interpretation**: P@5=0.6 means 60% of returned results are relevant

**Recall@K**
- **Best for**: Systems where completeness (low false negatives) is crucial  
- **Example**: Legal discovery where missing documents has consequences
- **Interpretation**: R@5=0.4 means we found 40% of all relevant documents

**F1@K**
- **Best for**: Balanced assessment of precision and recall
- **Example**: General-purpose search systems
- **Interpretation**: F1@5=0.5 balances finding relevant docs with avoiding irrelevant ones

### Choosing the Right Metric for Your Use Case

| Use Case | Primary Metric | Reasoning |
|----------|---------------|-----------|
| **QA Systems** | MRR | Users need one good answer fast |
| **Research/Discovery** | NDCG@10 | Users explore multiple results |
| **Fact Verification** | Precision@5 | Accuracy more important than completeness |
| **Legal/Compliance** | Recall@10 | Can't afford to miss relevant documents |
| **General Search** | F1@5 or NDCG@5 | Balance of multiple factors |

### Statistical Significance Testing

For production systems, always test statistical significance:
- Use paired t-tests to compare methods
- Require p < 0.05 for claiming improvements  
- Test on diverse query sets (easy + hard queries)
- Consider effect size, not just statistical significance